In [1]:
from datetime import datetime, date, timedelta
import sys
import tkinter as tk
from tkinter import ttk, messagebox
import requests
import browser_cookie3
import pandas as pd
import PublicDataReader as pdr
import json
from urllib.parse import quote
import threading
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import requests
import io
import os
import time
import random
import webbrowser
import tkinter.filedialog

# 새로 추가된 시계열 다운로드 관련 import
import shutil
import glob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from bs4 import BeautifulSoup # 맨 위 import 부분에 추가해 주세요.

IS_WINDOWS = os.name == 'nt'
if IS_WINDOWS:
    try:
        import win32gui
        import win32con
        print("pywin32 라이브러리를 성공적으로 로드했습니다.")
    except ImportError:
        print("경고: pywin32 라이브러리가 설치되지 않았습니다. 'pip install pywin32'를 실행하여 설치하세요.")
        IS_WINDOWS = False

# Tkinter 고해상도(High DPI) 지원 시도
if IS_WINDOWS:
    try:
        from ctypes import windll
        windll.shcore.SetProcessDpiAwareness(1)
        print("DPI 인식 설정을 시도했습니다.")
    except:
        print("DPI 인식 설정 중 오류가 발생했거나 지원되지 않는 환경입니다.")
        pass

# 시계열 다운로드 설정 변수
TIMESERIES_DOWNLOAD_DIR = r"C:\Bi시각화\시계열"
KB_BASE_URL = "https://kbland.kr/webview.html#/main/statistics?channel=kbland"
KB_WEEKLY_TAB_XPATH = '//*[@id="__BVID__30___BV_tab_button__"]'
DOWNLOAD_BUTTON_XPATH = '//*[@id="reference2"]/div[1]/button'
INITIAL_LOAD_WAIT = 3
TAB_LOAD_WAIT = 3

# 매물수집기 관련 상수들
_BUILD_ID_ = "KVZ8_AwgDYT1YkrfeHfcs"
EXPIRATION_DATE = date(2025, 12, 31)

# API 엔드포인트
REGION_API_BASE_URL = f"https://fin.land.naver.com/_next/data/{_BUILD_ID_}/regions.json" 
GET_API_URL = "https://m.land.naver.com/cluster/ajax/articleList" 
GET_REGION_LIST_API_URL = "https://m.land.naver.com/map/getRegionList" 

# 매물수집기용 헤더 설정
NAVER_HEADERS = {
    "Host": "m.land.naver.com",
    "User-Agent": "A",
}

class InvestmentTableProgram:
    def __init__(self):
        
        self.root = tk.Tk()
        self.root.title("DAON_Searcher")

        # Matplotlib 폰트 설정
        try:
            font_name = fm.FontProperties(family='Malgun Gothic').get_name()
            plt.rcParams['font.family'] = font_name
            plt.rcParams['axes.unicode_minus'] = False
        except Exception as e:
            messagebox.showwarning("폰트 설정 오류", f"폰트 설정 중 오류 발생: {e}\n기본 폰트로 대체합니다.")
            plt.rcParams['font.family'] = 'DejaVu Sans'

        self.root.state('zoomed')

        # 페이지 구성 관리
        self.page_order = [1, 2, 4, 3, 5]
        self.page_names = {
            1: "투자테이블", 2: "시세차트", 3: "시계열분석", 4: "매물수집기", 5: "페이지5"
        }
        self.current_page = 1 
        self.current_page_frame = None

        # 필터 설정 변수들
        self.region_var = tk.StringVar(value="4481000000") 
        self.sido_var = tk.StringVar()                     
        self.sigungu_var = tk.StringVar()                  
        self.region_data = {}                              

        # 날짜, 면적, 증감율 등 기타 필터 변수
        self.start_date_var = tk.StringVar()
        self.end_date_var = tk.StringVar()
        end_date = datetime.now()
        start_date = end_date - pd.Timedelta(days=180)
        self.start_date_var.set(start_date.strftime("%Y-%m-%d"))
        self.end_date_var.set(end_date.strftime("%Y-%m-%d"))
        self.exclusive_area_min = tk.StringVar(value="0")
        self.exclusive_area_max = tk.StringVar(value="999")
        self.max_years = tk.StringVar(value="30")
        self.max_unit_price = tk.StringVar(value="999999")
        self.rate_min = tk.StringVar(value="-100")
        self.rate_max = tk.StringVar(value="100")

        # 데이터 저장 변수
        self.raw_data = None
        self.filtered_data = None
        self.selected_apartments = {}
        self.all_fetched_articles = [] # 모든 원본 매물 데이터를 저장할 변수

        # 시계열 분석 관련 변수 추가
        self.timeseries_download_path_var = tk.StringVar()
        self.set_default_timeseries_download_path()
        self.timeseries_sido_var = tk.StringVar()
        self.timeseries_sigungu_var = tk.StringVar()
        self.timeseries_period_start_var = tk.StringVar()
        self.timeseries_period_end_var = tk.StringVar()
        
        # 차트 데이터 선택 변수들 (새로 추가)
        self.chart_data_vars = {
            '매수심리': tk.BooleanVar(value=True),  # 초기에 선택됨
            '전세현황': tk.BooleanVar(value=False),
            '전세수급': tk.BooleanVar(value=False),
            '매매전망': tk.BooleanVar(value=False),
            '전세전망': tk.BooleanVar(value=False),
            '매매지수': tk.BooleanVar(value=False),
            '전세지수': tk.BooleanVar(value=False),
            '매매증감': tk.BooleanVar(value=False),
            '전세증감': tk.BooleanVar(value=False)
        }
        
        # 기본 기간 설정 (최근 1년)
        end_date_ts = datetime.now()
        start_date_ts = end_date_ts - pd.Timedelta(days=365)
        self.timeseries_period_start_var.set(start_date_ts.strftime("%Y-%m-%d"))
        self.timeseries_period_end_var.set(end_date_ts.strftime("%Y-%m-%d"))
        
        # 시계열 차트 관련 변수
        self.timeseries_chart_frame = None
        self.timeseries_fig = None
        self.timeseries_ax = None
        self.timeseries_canvas = None
        self.current_timeseries_data = None

        # 엑셀 내보내기 경로 저장 변수 추가
        self.export_path_var = tk.StringVar()
        self.set_default_export_path()

        # 차트 관련 변수
        self.fig = None; self.ax = None; self.ax2 = None; self.canvas = None
        self.chart_info_labels = {}; self.current_chart_data = None
        self.current_selected_apt_info = None

        # 차트 옵션 변수
        self.show_sise_sale_var = tk.BooleanVar(value=True)
        self.show_sise_lease_var = tk.BooleanVar(value=True)
        self.show_sise_ratio_var = tk.BooleanVar(value=True)
        self.show_real_sale_var = tk.BooleanVar(value=True)
        self.show_real_lease_var = tk.BooleanVar(value=True)
        self.chart_start_date_var = tk.StringVar()
        self.chart_end_date_var = tk.StringVar()

        # 실거래 리스트용 Treeviews
        self.sale_real_tran_tree = None
        self.lease_real_tran_tree = None

        # 투자테이블 Treeview 전용 정렬 방향 저장 딕셔너리
        self.investment_table_sort_directions = {}

        # 매물수집기 관련 변수들
        self._stop_flag = False
        self._regions_data = {} 
        self.fetched_article_data = []

        # 매물 개수를 표시할 StringVar 추가 ---
        self.collector_sale_count_var = tk.StringVar(value="매매: 0개")
        self.collector_lease_count_var = tk.StringVar(value="전세: 0개")
        self.collector_monthly_count_var = tk.StringVar(value="월세: 0개")
        self.collector_total_count_var = tk.StringVar(value="총: 0개") # 총 매물 수도 추가하면 좋을 것 같습니다.

        # 단지명 필터링을 위한 변수 추가
        self.all_fetched_articles = [] # API에서 가져온 모든 원본 매물 데이터를 저장
        self.collector_complex_name_var = tk.StringVar(value="전체") # 콤보박스 선택값 저장
        self.unique_complex_names = [] # 고유 단지명 목록 저장 (콤보박스에 표시될 값)

        # 거래유형 필터링을 위한 변수 추가 ---
        self.collector_trad_type_var = tk.StringVar(value="전체") # 콤보박스 선택값 저장
        self.unique_trad_types = [] # 고유 거래유형 목록 저장

        # 지역코드.txt 로드 메서드 호출
        self.load_region_data() 

        # UI 레이아웃 핵심
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # 내비게이션 바
        self.create_navigation_bar(self.main_frame) 

        # 페이지 내용이 표시될 컨텐츠 프레임
        self.content_frame = ttk.Frame(self.main_frame)
        self.content_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))

        # 로딩 레이블 (UI 하단에 표시될 메인 상태바 역할)
        self.loading_label = ttk.Label(self.main_frame, text="프로그램 준비 완료", font=('Arial', 12))
        self.loading_label.pack(side=tk.BOTTOM, fill=tk.X, pady=5)

        # debug_label을 loading_label과 같은 객체로 사용하여,
        # update_debug_label 호출 시 loading_label의 텍스트가 업데이트되도록 합니다.
        self.debug_label = self.loading_label

        # 페이지 프레임 생성 및 UI 그리기 호출
        self.page_frames = {} 
        
        # 1페이지 (투자테이블) 프레임 생성 및 UI 그리기
        self.page_frames[1] = ttk.Frame(self.content_frame)
        self.create_investment_table_page(self.page_frames[1])

        # 2페이지 (시세차트) 프레임 생성 및 UI 그리기 - 매물 검색 기능 제거됨
        self.page_frames[2] = ttk.Frame(self.content_frame)
        self.create_chart_page(self.page_frames[2])

        # 3페이지 (시계열분석) 프레임 생성 및 UI 그리기
        self.page_frames[3] = ttk.Frame(self.content_frame)
        self.create_timeseries_analysis_page(self.page_frames[3])

        # 4페이지 (매물수집기) - 새로 추가
        self.page_frames[4] = ttk.Frame(self.content_frame)
        self.create_property_collector_page(self.page_frames[4])
        
        # 5페이지는 임시 페이지
        self.page_frames[5] = ttk.Frame(self.content_frame)
        ttk.Label(self.page_frames[5], text="5페이지 입니다. (임시)", font=('Arial', 20)).pack(pady=50)

        # 시계열 다운로드 폴더 생성 확인
        self.ensure_timeseries_download_dir()

        # 프로그램 시작 시 초기 페이지 로드
        self.load_page(1)

    # User-Agent 기본 헤더 설정
    DEFAULT_HEADERS = {
        "User-Agent": "A",
        "Referer": "https://new.land.naver.com/",
        "Origin": "https://new.land.naver.com",
    }

    def set_default_timeseries_download_path(self):
        """시계열 다운로드 기본 경로 설정"""
        self.timeseries_download_path_var.set(TIMESERIES_DOWNLOAD_DIR)

    def ensure_timeseries_download_dir(self):
        """시계열 다운로드 폴더 생성 확인"""
        download_path = self.timeseries_download_path_var.get()
        if not os.path.exists(download_path):
            try:
                os.makedirs(download_path)
                print(f"시계열 다운로드 폴더를 생성했습니다: {download_path}")
            except OSError as e:
                messagebox.showerror("오류", f"시계열 다운로드 폴더 생성 실패: {e}")

    def show_error(self, error_msg):
        """일반 오류 메시지를 표시하는 범용 메서드"""
        self.loading_label.config(text="오류가 발생했습니다.")
        messagebox.showerror("오류", error_msg)        

    def update_debug_label(self, message):
        """디버깅 라벨 텍스트를 업데이트하는 메서드 (loading_label을 재활용)"""
        if self.debug_label: # debug_label이 초기화되었는지 확인
           # loading_label의 텍스트를 업데이트합니다.
           # 다른 메시지와 구분하기 위해 "[DEBUG]" 접두어를 추가했습니다.
           self.debug_label.config(text=f"[DEBUG] {message}")
           self.root.update_idletasks() # UI 업데이트를 강제합니다.

    def fetch_naver_api_data(self, url):
        """
        네이버 부동산 API에서 데이터를 가져오는 범용 메서드
        - 브라우저에서 복사한 쿠키를 하드코딩
        - browser_cookie3 없이 동작
        """
        try:
            headers = {
                'referer': 'https://fin.land.naver.com/',
                'user-agent': (A
                ) }

            session = requests.Session()
            session.headers.update(headers)

            # 1~3초 랜덤 대기 (429 방지)
            time.sleep(random.uniform(5, 10))

            response = session.get(url, timeout=10)
            response.raise_for_status()

            if response.headers.get('content-type', '').startswith('application/json'):
                return response.json()
            else:
                print(f"경고: JSON 응답이 아님. Content-Type: {response.headers.get('content-type')}")
                print(response.text[:500])
                return None

        except requests.exceptions.RequestException as e:
            print(f"API 요청 오류: {e}")
            if e.response is not None:
                print(f"응답 코드: {e.response.status_code}")
                print(e.response.text[:500])
            return None
        except Exception as e:
            print(f"예상치 못한 오류 발생: {e}")
            return None
            
    def set_default_export_path(self):
        """Excel 내보내기 기본 경로 설정"""
        home_dir = os.path.expanduser("~")
        download_dir = os.path.join(home_dir, "Downloads")
        self.export_path_var.set(download_dir)

    def load_region_data(self):
        """지역 코드 파일 로드 및 파싱 (콤보박스 등에 필요)"""
        # GitHub Raw URL
        github_raw_url = "https://raw.githubusercontent.com/kaga-fo/DaonSearcher_Project/refs/heads/main/%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C.txt"
        
        # 로컬 파일 경로
        local_fallback_path = r"C:\Users\kagaj\code\지역코드.txt"
        
        self.region_data = {}

        file_source_name = ""
        file_content_object = None

        try:
            # 1. GitHub에서 로드 시도
            file_source_name = "GitHub"
            print(f"[{file_source_name}]에서 지역코드 파일 로드 시도: {github_raw_url}")
            response = requests.get(github_raw_url, timeout=10)
            response.raise_for_status()

            file_content_object = io.StringIO(response.text)
            print(f"[{file_source_name}]에서 지역코드 파일 성공적으로 로드.")

        except requests.exceptions.RequestException as e:
            # 2. GitHub 로드 실패 시 로컬 파일 시도
            print(f"[{file_source_name}]에서 지역코드 파일 로드 중 오류 발생: {e}. 로컬 파일로 대체 시도.")
            if os.path.exists(local_fallback_path):
                file_source_name = "로컬 파일"
                try:
                    file_content_object = open(local_fallback_path, 'r', encoding='utf-8-sig') 
                    print(f"[{file_source_name}] '{local_fallback_path}' 성공적으로 로드.")
                except Exception as ex:
                    messagebox.showerror("오류", f"로컬 지역코드 파일 로드 중 오류 발생: {str(ex)}\n파일 인코딩 또는 경로를 확인해주세요.")
                    self.region_data = {}
                    return
            else:
                messagebox.showerror("오류", f"지역코드 파일이 없습니다.\nGitHub에서도 로컬({local_fallback_path})에서도 찾을 수 없습니다.")
                self.region_data = {}
                return
        except Exception as e:
            messagebox.showerror("오류", f"지역코드 파일 처리 중 알 수 없는 오류 발생: {str(e)}")
            self.region_data = {}
            return

        # 파싱 로직
        try:
            for line in file_content_object:
                line = line.strip()
                if not line:
                    continue
                parts = line.split()
                
                if len(parts) >= 3: 
                    code = parts[0]
                    sido = parts[1]
                    sigungu = parts[2]
                    if sido not in self.region_data:
                        self.region_data[sido] = {}
                    self.region_data[sido][sigungu] = code
                elif len(parts) == 2: 
                    code = parts[0]
                    sido = parts[1]
                    if sido not in self.region_data:
                        self.region_data[sido] = {}
                    self.region_data[sido][''] = code
                else:
                    print(f"경고: 지역코드 파일 '{file_source_name}'의 형식이 올바르지 않습니다 (최소 2개 항목 필요): {line}")

            print(f"지역코드 파싱 완료. 총 {len(self.region_data)}개의 시도 데이터.")
        except Exception as e:
            messagebox.showerror("오류", f"지역코드 파일 파싱 중 오류 발생: {str(e)}")
            self.region_data = {}
        finally:
            if hasattr(file_content_object, 'close'): 
                file_content_object.close()

    # 매물수집기 관련 메서드들
    
    # 부동산 유형 코드 매핑 (한글 이름 -> API 코드)
    _RLET_TYPE_CODES = {
        '아파트': 'APT',
        '아파트분양권': 'ABYG',
        '빌라': 'VL:YR:DSD',  # 빌라 선택 시 빌라, 연립, 다세대 모두 포함
        '오피스텔': 'OPST',
    }

    # 거래 유형 코드 매핑 (한글 이름 -> API 코드)
    _TRAD_TYPE_CODES = {
        '매매': 'A1',
        '전세': 'B1',
        '월세': 'B2',
    }

    # 평형대 코드 및 spcMin/spcMax 매핑 (사용자 표시명 -> {spcMin, spcMax})
    _PYEONG_TYPE_RANGES = {
        '~10평': {'spcMax': 33},
        '10평~20평': {'spcMin': 33, 'spcMax': 66},
        '20평~30평': {'spcMin': 66, 'spcMax': 99},
        '30평~40평': {'spcMin': 99, 'spcMax': 132},
        '40평~50평': {'spcMin': 132, 'spcMax': 165},
        '50평~60평': {'spcMin': 165, 'spcMax': 198},
        '60평~70평': {'spcMin': 198, 'spcMax': 231},
        '70평~': {'spcMin': 231, 'spcMax': 900000000}, # 매우 큰 값으로 무제한 의미
    }

    # 줌 레벨에 따른 대략적인 델타 값 (시행착오를 통해 조정 필요)
    _Z_LEVEL_DELTAS = {
        10: {'lat_delta': 0.2, 'lon_delta': 0.2},
        11: {'lat_delta': 0.1, 'lon_delta': 0.1},
        12: {'lat_delta': 0.05, 'lon_delta': 0.05}, 
        13: {'lat_delta': 0.025, 'lon_delta': 0.025},
        14: {'lat_delta': 0.012, 'lon_delta': 0.012},
        15: {'lat_delta': 0.006, 'lon_delta': 0.006},
        16: {'lat_delta': 0.003, 'lon_delta': 0.003},
    }

    def update_status(self, message):
        """상태 메시지 업데이트"""
        if hasattr(self, 'status_label'):
            self.status_label.config(text=message)
            self.root.update_idletasks()
        else:
            self.loading_label.config(text=message)
            self.root.update_idletasks()

    def _safe_request(self, method, url, headers, params=None, json_data=None, timeout=10, api_name="API"):
        try:
            print(f"\n--- API 요청 시작: {api_name} ---")
            print(f"  URL: {url}")
            print(f"  Method: {method.upper()}")
            print(f"  Params: {params}")
            
            if method.lower() == 'post':
                response = requests.post(url, headers=headers, json=json_data, timeout=timeout)
            elif method.lower() == 'get':
                response = requests.get(url, headers=headers, params=params, timeout=timeout)
            else:
                raise ValueError("지원하지 않는 HTTP 메서드입니다.")

            response.raise_for_status() 

            print(f"  Status Code: {response.status_code}")
            return response.json()
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                delay_time = random.uniform(5, 15)
                self.update_status(f"⚠️ {api_name} 429 오류 발생. {delay_time:.2f}초 후 재시도.")
                time.sleep(delay_time)
                return "RETRY"
            else:
                response_text = e.response.text if hasattr(e.response, 'text') else '응답 본문 없음'
                self.root.after(0, lambda e_val=e, res_text_val=response_text: messagebox.showerror("HTTP 오류", f"{api_name} 요청 중 오류 발생:\n상태코드: {e_val.response.status_code}\nURL: {e_val.request.url}\n응답: {res_text_val}"))
                self.update_status(f"⛔️ {api_name} HTTP 오류: {e.response.status_code}")
        except requests.exceptions.ConnectionError as e:
            self.root.after(0, lambda e_val=e: messagebox.showerror("연결 오류", f"{api_name} 서버 연결 실패:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 연결 오류: {e}")
        except requests.exceptions.Timeout as e:
            self.root.after(0, lambda e_val=e: messagebox.showerror("시간 초과", f"{api_name} 요청 시간 초과:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 시간 초과: {e}")
        except requests.exceptions.RequestException as e:
            self.root.after(0, lambda e_val=e: messagebox.showerror("요청 오류", f"{api_name} 요청 중 알 수 없는 오류:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 알 수 없는 오류: {e}")
        except json.JSONDecodeError as e:
            response_text = response.text if 'response' in locals() else '없음'
            self.root.after(0, lambda e_val=e, res_text_val=response_text: messagebox.showerror("JSON 파싱 오류", f"{api_name} 응답 JSON 디코딩 실패:\n{e_val}\n응답 본문: {res_text_val}"))
            self.update_status(f"⛔️ {api_name} JSON 파싱 오류: {e}")
        except Exception as e:
            self.root.after(0, lambda e_val=e: messagebox.showerror("알 수 없는 오류", f"{api_name} 예상치 못한 오류:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 예상치 못한 오류: {e}")
        return None

    # 다운로드 관련 메서드들

    def wait_for_download_completion(self, download_path, timeout=60):
        """다운로드 완료 대기 유틸리티 메서드"""
        print(f"다운로드 폴더 '{download_path}'에서 새로운 파일 다운로드를 기다립니다... (최대 {timeout}초)")
        seconds = 0
        last_size = -1
        file_path = None
        start_time = time.time()

        existing_files = {os.path.join(download_path, f): os.path.getctime(os.path.join(download_path, f))
                          for f in os.listdir(download_path) if os.path.isfile(os.path.join(download_path, f))}

        while time.time() - start_time < timeout:
            time.sleep(1)
            seconds += 1

            list_of_files = [os.path.join(download_path, f) for f in os.listdir(download_path) if os.path.isfile(os.path.join(download_path, f))]

            if not list_of_files:
                 continue

            new_files = [(f, os.path.getctime(f)) for f in list_of_files if f not in existing_files]

            if new_files:
                latest_new_file, latest_ctime = max(new_files, key=lambda item: item[1])

                if not latest_new_file.endswith('.crdownload'):
                     file_path = latest_new_file
                     current_size = os.path.getsize(file_path)

                     if current_size > 0 and current_size == last_size and seconds > 2:
                         print(f"파일 크기 안정됨: {current_size} bytes. 다운로드 완료 추정.")
                         return file_path
                     else:
                         last_size = current_size

        print(f"다운로드 완료 대기 시간 초과 ({timeout}초). 새로운 파일이 감지되지 않았거나 다운로드가 완료되지 않았습니다.")
        return None

    def process_downloaded_file(self, downloaded_file_path, target_name):
        """다운로드된 파일을 복사하고 원하는 이름으로 변경하여 저장"""
        if not downloaded_file_path or not os.path.exists(downloaded_file_path):
            print("다운로드된 파일을 찾을 수 없습니다.")
            return False

        download_path = self.timeseries_download_path_var.get()
        target_file_name = f"{target_name}.xlsx"
        target_file_path = os.path.join(download_path, target_file_name)

        print(f"다운로드된 원본 파일: {os.path.basename(downloaded_file_path)}")
        print(f"복사본 대상 파일명: {target_file_name}")

        # 기존에 같은 이름의 대상 파일이 있으면 삭제
        if os.path.exists(target_file_path):
            print(f"기존 대상 파일 삭제 시도: {target_file_name}")
            try:
                os.remove(target_file_path)
                print("기존 파일 삭제 완료.")
            except OSError as e:
                print(f"기존 대상 파일 삭제 실패: {e}")
                messagebox.showerror("오류", f"기존 파일 삭제 실패: {target_file_name}\n{e}\n파일이 사용 중인지 확인해 보세요.")
                return False

        # 다운로드된 파일을 원하는 이름으로 복사
        try:
            shutil.copy2(downloaded_file_path, target_file_path)
            print(f"파일 복사 완료: {os.path.basename(downloaded_file_path)} -> {target_file_name}")
            return True
        except OSError as e:
            print(f"파일 복사 실패: {e}")
            messagebox.showerror("오류", f"파일 복사 실패: {os.path.basename(downloaded_file_path)} -> {target_file_name}\n{e}")
            return False

    def download_kb_weekly_timeseries(self):
        """KB 주간 시계열 다운로드 (버튼 command)"""
        self.loading_label.config(text="KB 주간 시계열 다운로드를 시작합니다...")
        
        # 스레드에서 실행하여 UI가 멈추지 않도록 함
        thread = threading.Thread(target=self._run_kb_weekly_download)
        thread.daemon = True
        thread.start()

    def download_kb_monthly_timeseries(self):
        """KB 월간 시계열 다운로드 (버튼 command)"""
        self.loading_label.config(text="KB 월간 시계열 다운로드를 시작합니다...")
        
        # 스레드에서 실행하여 UI가 멈추지 않도록 함
        thread = threading.Thread(target=self._run_kb_monthly_download)
        thread.daemon = True
        thread.start()

    def _run_kb_weekly_download(self):
        """KB 주간 시계열 다운로드 실행 (스레드 타겟)"""
        driver = None
        downloaded_file_path = None
        
        try:
            download_path = self.timeseries_download_path_var.get()
            self.ensure_timeseries_download_dir()

            # Chrome 옵션 설정
            chrome_options = webdriver.ChromeOptions()
            prefs = {
                "download.default_directory": download_path,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            }
            chrome_options.add_experimental_option("prefs", prefs)
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")

            print("KB 주간 시계열 다운로드 시작...")
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

            print(f"페이지 접속: {KB_BASE_URL}")
            driver.get(KB_BASE_URL)

            # 페이지 로딩 및 초기화 대기
            print(f"페이지 로딩 및 초기화 대기 ({INITIAL_LOAD_WAIT}초)...")
            time.sleep(INITIAL_LOAD_WAIT)

            # '주간시계열' 탭 클릭
            try:
                print(f"주간시계열 탭 찾기 및 클릭 시도: {KB_WEEKLY_TAB_XPATH}")
                weekly_tab = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, KB_WEEKLY_TAB_XPATH))
                )
                weekly_tab.click()
                print("주간시계열 탭 클릭 완료.")
                print(f"탭 내용 로딩 및 다운로드 버튼 활성화 대기 ({TAB_LOAD_WAIT}초)...")
                time.sleep(TAB_LOAD_WAIT)

            except (NoSuchElementException, TimeoutException) as e:
                error_msg = f"주간시계열 탭을 찾거나 클릭할 수 없습니다.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return
            except Exception as e:
                error_msg = f"주간시계열 탭 클릭 중 오류 발생.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return

            # 다운로드 버튼 클릭
            try:
                print(f"다운로드 버튼 찾기 및 클릭 시도: {DOWNLOAD_BUTTON_XPATH}")
                download_button = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, DOWNLOAD_BUTTON_XPATH))
                )
                download_button.click()
                print("다운로드 버튼 클릭 완료.")
            except (NoSuchElementException, TimeoutException) as e:
                error_msg = f"다운로드 버튼을 찾거나 클릭할 수 없습니다.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return
            except Exception as e:
                error_msg = f"다운로드 버튼 클릭 중 오류 발생.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return

            # 파일 다운로드 완료 대기
            print("파일 다운로드 완료 대기...")
            downloaded_file_path = self.wait_for_download_completion(download_path, timeout=45)

            if downloaded_file_path:
                print("다운로드된 파일 확인됨. 파일 처리 시작...")
                success = self.process_downloaded_file(downloaded_file_path, "주간시계열")
                if success:
                    self.root.after(0, lambda: messagebox.showinfo("완료", "KB 주간 시계열 다운로드 완료"))
                    self.root.after(0, lambda: self.loading_label.config(text="KB 주간 시계열 다운로드 완료"))
                else:
                    self.root.after(0, lambda: self.loading_label.config(text="파일 처리 실패"))
            else:
                error_msg = "KB 주간 시계열 다운로드 중 시간 초과 또는 파일 감지 실패."
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        except WebDriverException as e:
            error_msg = f"웹 브라우저 제어 중 오류 발생.\n{e}"
            self.root.after(0, lambda: messagebox.showerror("WebDriver 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))
        except Exception as e:
            error_msg = f"KB 주간 시계열 다운로드 중 오류 발생: {e}"
            self.root.after(0, lambda: messagebox.showerror("알 수 없는 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        finally:
            if driver:
                driver.quit()
                print("브라우저 종료.")
            print("KB 주간 시계열 다운로드 프로세스 종료.")

    def _run_kb_monthly_download(self):
        """KB 월간 시계열 다운로드 실행 (스레드 타겟)"""
        driver = None
        downloaded_file_path = None

        try:
            download_path = self.timeseries_download_path_var.get()
            self.ensure_timeseries_download_dir()

            # Chrome 옵션 설정
            chrome_options = webdriver.ChromeOptions()
            prefs = {
                "download.default_directory": download_path,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            }
            chrome_options.add_experimental_option("prefs", prefs)
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")

            print("KB 월간 시계열 다운로드 시작...")
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

            print(f"페이지 접속: {KB_BASE_URL}")
            driver.get(KB_BASE_URL)

            # 페이지 로딩 및 초기화 대기
            print(f"페이지 로딩 및 초기화 대기 ({INITIAL_LOAD_WAIT}초)...")
            time.sleep(INITIAL_LOAD_WAIT)

            # 다운로드 버튼 클릭 (월간시계열은 기본 탭이므로 탭 클릭 불필요)
            try:
                print(f"다운로드 버튼 찾기 및 클릭 시도: {DOWNLOAD_BUTTON_XPATH}")
                download_button = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, DOWNLOAD_BUTTON_XPATH))
                )
                download_button.click()
                print("다운로드 버튼 클릭 완료.")

            except (NoSuchElementException, TimeoutException) as e:
                error_msg = f"다운로드 버튼을 찾거나 클릭할 수 없습니다.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return
            except Exception as e:
                error_msg = f"다운로드 버튼 클릭 중 오류 발생.\n{e}"
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                return

            # 파일 다운로드 완료 대기
            print("파일 다운로드 완료 대기...")
            downloaded_file_path = self.wait_for_download_completion(download_path, timeout=45)

            if downloaded_file_path:
                print("다운로드된 파일 확인됨. 파일 처리 시작...")
                success = self.process_downloaded_file(downloaded_file_path, "월간시계열")
                if success:
                    self.root.after(0, lambda: messagebox.showinfo("완료", "KB 월간 시계열 다운로드 완료"))
                    self.root.after(0, lambda: self.loading_label.config(text="KB 월간 시계열 다운로드 완료"))
                else:
                    self.root.after(0, lambda: self.loading_label.config(text="파일 처리 실패"))
            else:
                error_msg = "KB 월간 시계열 다운로드 중 시간 초과 또는 파일 감지 실패."
                self.root.after(0, lambda: messagebox.showerror("오류", error_msg))
                self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        except WebDriverException as e:
            error_msg = f"웹 브라우저 제어 중 오류 발생.\n{e}"
            self.root.after(0, lambda: messagebox.showerror("WebDriver 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))
        except Exception as e:
            error_msg = f"KB 월간 시계열 다운로드 중 오류 발생: {e}"
            self.root.after(0, lambda: messagebox.showerror("알 수 없는 오류", error_msg))
            self.root.after(0, lambda: self.loading_label.config(text="다운로드 실패"))

        finally:
            if driver:
                driver.quit()
                print("브라우저 종료.")
            print("KB 월간 시계열 다운로드 프로세스 종료.")

    # UI 생성 메서드들

    def create_navigation_bar(self, parent):
        """메인 내비게이션 바 생성"""
        nav_frame = ttk.Frame(parent)
        nav_frame.pack(fill=tk.X, pady=(0, 10))

        ttk.Button(nav_frame, text="페이지 순서 변경", command=self.change_page_order).pack(side=tk.LEFT, padx=(0, 20))

        self.page_buttons = {}
        for i, page_num in enumerate(self.page_order):
            btn = ttk.Button(nav_frame, text=f"{i+1}. {self.page_names[page_num]}", command=lambda p=page_num: self.load_page(p))
            btn.pack(side=tk.LEFT, padx=5)
            self.page_buttons[page_num] = btn

    def change_page_order(self):
        """페이지 순서 변경 (임시 구현)"""
        messagebox.showinfo("정보", "페이지 순서 변경 기능은 향후 구현 예정입니다.")

    def load_page(self, page_num):
        """페이지 전환 및 로드"""
        # 현재 화면에 보이는 페이지 프레임이 있다면 숨김
        if self.current_page_frame:
            self.current_page_frame.pack_forget() # 이전 페이지 숨기기
        
        self.current_page = page_num # 현재 페이지 번호 업데이트
        self.current_page_frame = self.page_frames[page_num] # 새 페이지 프레임 가져오기
        self.current_page_frame.pack(fill=tk.BOTH, expand=True) # 새 페이지 표시
        
        # ===== 이 조건문 추가 =====
        # 시세차트 (2페이지)로 전환될 때만 '선택 단지 목록' Treeview를 업데이트
        if page_num == 2: # 시세차트 페이지의 페이지 번호는 2로 설정되어 있습니다.
            self._update_selected_apt_tree() # 새로 추가한 Treeview 업데이트 메서드 호출
        # ========================

        # 버튼 상태 업데이트 (현재 페이지 버튼 비활성화, 다른 버튼 활성화)
        for p, btn in self.page_buttons.items():
            if p == page_num:
                btn.config(state=tk.DISABLED)
            else:
                btn.config(state=tk.NORMAL)

    def create_investment_table_page(self, parent_frame):
        """1페이지: 투자테이블 UI 생성"""
        title_label = ttk.Label(parent_frame, text="투자테이블", font=('Arial', 16, 'bold')) 
        title_label.pack(pady=(0, 20))

        filter_frame = ttk.LabelFrame(parent_frame, text="필터 설정", padding=10) 
        filter_frame.pack(fill=tk.X, pady=(0, 10))

        self.create_filter_ui(filter_frame)

        result_frame = ttk.LabelFrame(parent_frame, text="조회 결과", padding=10)
        result_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))

        self.create_result_table(result_frame)

##    def create_investment_table_page(self, parent_frame):
##        """1페이지: 투자테이블 UI 생성"""
##        title_label = ttk.Label(parent_frame, text="투자테이블", font=('Arial', 16, 'bold')) 
##        title_label.pack(pady=(0, 20))
    
##        filter_frame = ttk.LabelFrame(parent_frame, text="필터 설정", padding=10) 
##        filter_frame.pack(fill=tk.X, pady=(0, 10))
##        self.create_filter_ui(filter_frame)
    
##        result_frame = ttk.LabelFrame(parent_frame, text="조회 결과", padding=10)
##        result_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))
    
##        # 투자테이블 전용 Treeview 생성
##        columns = ('지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율','매매시세', '전세시세', '전세가율', '매전갭', '세대수')
##        self.invest_tree = ttk.Treeview(result_frame, columns=columns, show="headings")
##        self.invest_tree.pack(fill=tk.BOTH, expand=True)
    
##        for col in columns:
##            self.invest_tree.heading(col, text=col,
##                command=lambda _col=col: self.treeview_sort_column(self.invest_tree, _col, False))
    
##        # 스크롤바 추가
##        scrollbar = ttk.Scrollbar(result_frame, orient="vertical", command=self.invest_tree.yview)
##        self.invest_tree.configure(yscrollcommand=scrollbar.set)
##        scrollbar.pack(side="right", fill="y")


    def create_filter_ui(self, parent):
        """투자테이블 필터 UI 생성"""
        row = 0
        padx_val = 3
        pady_val = 3

        # 컬럼 가중치 설정 (8개 열)
        parent.grid_columnconfigure(0, weight=0, minsize=40)
        parent.grid_columnconfigure(1, weight=1)             
        parent.grid_columnconfigure(2, weight=0, minsize=40)
        parent.grid_columnconfigure(3, weight=1)             
        parent.grid_columnconfigure(4, weight=0, minsize=20)
        parent.grid_columnconfigure(5, weight=1)
        parent.grid_columnconfigure(6, weight=1)
        parent.grid_columnconfigure(7, weight=0, minsize=40)

        # Row 0: 시도, 시군구
        ttk.Label(parent, text="시도:").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        self.sido_combobox = ttk.Combobox(parent, textvariable=self.sido_var, values=list(self.region_data.keys()), state="readonly", width=15)
        self.sido_combobox.grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        self.sido_combobox.bind("<<ComboboxSelected>>", self.on_sido_selected)

        ttk.Label(parent, text="시군구:").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        self.sigungu_combobox = ttk.Combobox(parent, textvariable=self.sigungu_var, state="readonly", width=15)
        self.sigungu_combobox.grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)

        if self.region_data:
            first_sido = list(self.region_data.keys())[0]
            self.sido_var.set(first_sido)
            self.on_sido_selected(None)
        else:
            self.sido_combobox.config(state="disabled")
            self.sigungu_combobox.config(state="disabled")
            messagebox.showwarning("경고", "지역코드 파일 로드에 실패하여 지역 선택 기능을 사용할 수 없습니다.")
        row += 1 

        # Row 1: 시작일, 종료일
        ttk.Label(parent, text="시작일 (YYYY-MM-DD):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.start_date_var, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="종료일 (YYYY-MM-DD):").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.end_date_var, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1 

        # Row 2: 전용면적 (최소/최대)
        ttk.Label(parent, text="전용면적 (최소):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.exclusive_area_min, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="전용면적 (최대):").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.exclusive_area_max, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1 

        # Row 3: 연차, 평단가
        ttk.Label(parent, text="연차 (최대):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.max_years, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="최대 평단가:").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.max_unit_price, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1 

        # Row 4: 증감율 (최소/최대)
        ttk.Label(parent, text="증감율 (최소%):").grid(row=row, column=0, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.rate_min, width=15).grid(row=row, column=1, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Label(parent, text="증감율 (최대%):").grid(row=row, column=2, sticky=tk.W, padx=padx_val, pady=pady_val)
        ttk.Entry(parent, textvariable=self.rate_max, width=15).grid(row=row, column=3, sticky=tk.W, padx=padx_val, pady=pady_val)
        row += 1

        # Row 5: 모든 버튼과 경로 UI 배치
        # 조회 버튼
        ttk.Button(parent, text="조회", command=self.search_data).grid(row=row, column=0, columnspan=2, pady=10)
    
        # Excel 내보내기 버튼
        ttk.Button(parent, text="Excel 내보내기", command=self.export_to_excel).grid(row=row, column=2, columnspan=2, pady=10)

        # 저장 경로 라벨
        ttk.Label(parent, text="저장 경로:", font=('Arial', 9)).grid(row=row, column=4, sticky=tk.W, padx=padx_val, pady=pady_val)
    
        # 저장 경로 표시 라벨
        self.path_display_label = ttk.Label(parent, textvariable=self.export_path_var, wraplength=180, justify=tk.LEFT, font=('Arial', 9))
        self.path_display_label.grid(row=row, column=5, columnspan=2, sticky="ew", padx=padx_val, pady=pady_val)
    
        # 경로 변경 버튼
        ttk.Button(parent, text="변경", command=self.choose_export_path, width=5).grid(row=row, column=7, sticky=tk.E, padx=padx_val, pady=pady_val)

    def on_sido_selected(self, event):
        """투자테이블 시도 선택 이벤트 핸들러"""
        selected_sido = self.sido_var.get()
        if selected_sido and selected_sido in self.region_data:
            sigungu_list = list(self.region_data[selected_sido].keys())
            self.sigungu_combobox['values'] = sigungu_list
            if sigungu_list:
                self.sigungu_var.set(sigungu_list[0])
            else:
                self.sigungu_var.set("")
        else:
            self.sigungu_combobox['values'] = []
            self.sigungu_var.set("")

    def search_data(self):
        """조회 버튼 클릭 시 데이터 검색 시작 (fetch_data 스레드 호출)"""
        selected_sido = self.sido_var.get()
        selected_sigungu = self.sigungu_var.get()

        if not self.region_data:
            messagebox.showwarning("경고", "지역코드 파일이 로드되지 않아 지역 선택 기능을 사용할 수 없습니다.")
            self.loading_label.config(text="조회 실패: 지역 데이터 없음")
            return

        if selected_sido and selected_sido in self.region_data:
            if selected_sigungu in self.region_data[selected_sido]:
                self.region_var.set(self.region_data[selected_sido][selected_sigungu])
            elif '' in self.region_data[selected_sido] and not selected_sigungu:
                self.region_var.set(self.region_data[selected_sido][''])
            else:
                messagebox.showwarning("경고", "유효한 시군구를 선택해주세요.")
                self.loading_label.config(text="조회 실패: 유효한 지역 선택 필요")
                return
        else:
            messagebox.showwarning("경고", "유효한 시도를 선택해주세요.")
            self.loading_label.config(text="조회 실패: 유효한 지역 선택 필요")
            return

        try:
            start_date_obj = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end_date_obj = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            if start_date_obj > end_date_obj:
                messagebox.showwarning("경고", "시작일은 종료일보다 빨라야 합니다.")
                self.loading_label.config(text="조회 실패: 날짜 범위 오류")
                return
        except ValueError:
            messagebox.showwarning("경고", "날짜 형식이 올바르지 않습니다 (YYYY-MM-DD).")
            self.loading_label.config(text="조회 실패: 날짜 형식 오류")
            return

        self.loading_label.config(text="데이터를 조회 중입니다...")
        thread = threading.Thread(target=self.fetch_data)
        thread.daemon = True
        thread.start()

    def export_to_excel(self):
        """Excel로 데이터 내보내기"""
        if not hasattr(self, 'filtered_data') or not self.filtered_data:
            messagebox.showwarning("경고", "내보낼 데이터가 없습니다. 먼저 조회를 실행하세요.")
            return
        try:
            df = pd.DataFrame(self.filtered_data)
            columns_order = ['지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율',
                             '매매시세', '전세시세', '전세가율', '매전갭', '세대수', '면적일련번호', '단지기본일련번호']
            df = df.reindex(columns=columns_order, fill_value='')

            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"투자테이블_{timestamp}.xlsx"

            save_directory = self.export_path_var.get()
            if not save_directory:
                self.set_default_export_path()
                save_directory = self.export_path_var.get()
            
            full_path = os.path.join(save_directory, filename)

            df.to_excel(full_path, index=False, engine='openpyxl')

            messagebox.showinfo("성공", f"데이터가 '{full_path}' 파일로 내보내기 완료되었습니다.")
        except Exception as e:
            messagebox.showerror("오류", f"Excel 내보내기 중 오류가 발생했습니다: {str(e)}")

    def choose_export_path(self):
        """저장 경로 선택"""
        path = tkinter.filedialog.askdirectory()
        if path:
            self.export_path_var.set(path)

    def fetch_data(self):
        """KB API에서 투자 데이터 가져오기 (update_table 호출)"""
        try:
            start_date_str = self.start_date_var.get()
            end_date_str = self.end_date_var.get()

            base_url = "https://api.kbland.kr/land-extra/price/v1/api/invstTblAptSearch"
            params = {
                "시작년월일": start_date_str.replace("-", ""),
                "종료년월일": end_date_str.replace("-", ""),
                "지역리스트": self.region_var.get()
            }
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            headers = {
                'User-Agent': 'A',
                #'Accept': 'application/json, text/plain, */*',
                #'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                #'Accept-Encoding': 'gzip, deflate, br',
                #'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = session.get(url, timeout=30)
                    response.raise_for_status()
                    break
                except requests.exceptions.ConnectionError as e:
                    if attempt == max_retries - 1:
                        raise e
                    time.sleep(2)

            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                raise ValueError(f"JSON 응답이 아닙니다. Content-Type: {response.headers.get('content-type')}")

            self.root.after(0, self.update_table, data)

        except requests.exceptions.ConnectionError as e:
            error_msg = f"연결 오류: API 서버에 연결할 수 없습니다. 네트워크 상태를 확인하거나 잠시 후 다시 시도해주세요."
            self.root.after(0, self.show_error, error_msg)
        except requests.exceptions.Timeout as e:
            error_msg = f"시간 초과: 요청이 시간 초과되었습니다. 잠시 후 다시 시도해주세요."
            self.root.after(0, self.show_error, error_msg)
        except requests.exceptions.HTTPError as e:
            error_msg = f"HTTP 오류: {e.response.status_code} - {e.response.reason}"
            self.root.after(0, self.show_error, error_msg)
        except ValueError as e:
            error_msg = f"데이터 형식 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)
        except Exception as e:
            error_msg = f"알 수 없는 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)

    def parse_item_data(self, item):
        """KB 투자 데이터 파싱"""
        try:
            region = item.get('지역명', '')
            data_info = item.get('데이터정보', {})

            complex_name = data_info.get('I2010', {}).get('단지명', '')
            area_info = data_info.get('I2020', {})
            exclusive_area = area_info.get('전용면적', '')
            household_count = data_info.get('I2030', {}).get('지수', '')
            years = data_info.get('I2040', {}).get('지수', '')

            unit_price_info = data_info.get('I2050', {})
            unit_price = unit_price_info.get('공급면적당지수', '')
            if isinstance(unit_price, str):
                unit_price = unit_price.replace(',', '')

            sale_info = data_info.get('I2060', {})
            sale_price = sale_info.get('지수', '')
            sale_rate = sale_info.get('증감', '')
            lease_info = data_info.get('I2070', {})
            lease_price = lease_info.get('지수', '')
            lease_rate = lease_info.get('증감', '')
            lease_ratio_info = data_info.get('I2080', {})
            lease_ratio = lease_ratio_info.get('지수', '')

            gap_info = data_info.get('I2090', {})
            gap_value = gap_info.get('지수', '')

            area_serial_no = item.get('면적일련번호', '')
            complex_serial_no = item.get('단지기본일련번호', '')

            return {
                '지역': region, '단지명': complex_name, '전용면적': exclusive_area, '연차': years,
                '평단가': unit_price, '매매증감율': sale_rate, '전세증감율': lease_rate,
                '매매시세': sale_price, '전세시세': lease_price, '전세가율': lease_ratio,
                '매전갭': gap_value, '세대수': household_count,
                '면적일련번호': area_serial_no, '단지기본일련번호': complex_serial_no
            }
        except Exception as e:
            print(f"데이터 파싱 중 오류 발생: {e}, 원본 아이템: {item}")
            return None

    def update_table(self, data):
        """투자테이블 Treeview 업데이트"""
        for item in self.tree.get_children():
            self.tree.delete(item)

        self.raw_data = data
        parsed_data = []

        items_to_parse = []
        if isinstance(data, dict) and 'dataBody' in data and \
           isinstance(data['dataBody'], dict) and 'data' in data['dataBody'] and \
           isinstance(data['dataBody']['data'], dict) and '데이터목록' in data['dataBody']['data']:
            items_to_parse = data['dataBody']['data']['데이터목록']

        for item in items_to_parse:
            parsed_item = self.parse_item_data(item)
            if parsed_item:
                parsed_data.append(parsed_item)

        self.filtered_data = self.filter_data(parsed_data)

        for i, item in enumerate(self.filtered_data):
            values = (
                item.get('지역', ''), item.get('단지명', ''), item.get('전용면적', ''),
                item.get('연차', ''), item.get('평단가', ''), item.get('매매증감율', ''),
                item.get('전세증감율', ''), item.get('매매시세', ''), item.get('전세시세', ''),
                item.get('전세가율', ''), item.get('매전갭', ''), item.get('세대수', '')
            )
            
            # === 색상 로직 적용 ===
            tags = []
            
            # 기본 zebra striping
            base_tag = 'evenrow' if i % 2 == 0 else 'oddrow'
            
            # 매매증감율과 전세증감율을 모두 고려한 우선순위 색상 적용
            sale_rate = None
            lease_rate = None
            lease_ratio = None
            
            try:
                # 매매증감율 파싱
                sale_rate_str = str(item.get('매매증감율', '0')).strip()
                if sale_rate_str and sale_rate_str not in ['', '-', '0']:
                    sale_rate = float(sale_rate_str)
                
                # 전세증감율 파싱
                lease_rate_str = str(item.get('전세증감율', '0')).strip()
                if lease_rate_str and lease_rate_str not in ['', '-', '0']:
                    lease_rate = float(lease_rate_str)
                
                # 전세가율 파싱
                lease_ratio_str = str(item.get('전세가율', '0')).strip()
                if lease_ratio_str and lease_ratio_str not in ['', '-', '0']:
                    lease_ratio = float(lease_ratio_str)
                    
            except (ValueError, TypeError):
                pass
            
            # 색상 우선순위 적용
            color_applied = False
            
            # 1순위: 전세가율이 90% 이상인 경우 (가장 중요한 지표)
            if lease_ratio is not None and lease_ratio >= 90:
                tags = ['high_ratio']
                color_applied = True
            
            # 2순위: 매매증감율이 매우 높거나 낮은 경우
            elif sale_rate is not None:
                if sale_rate >= 10:  # 매매증감율 10% 이상 상승
                    tags = ['positive_sale']
                    color_applied = True
                elif sale_rate <= -10:  # 매매증감율 10% 이상 하락
                    tags = ['negative_sale']
                    color_applied = True
            
            # 3순위: 전세증감율이 매우 높거나 낮은 경우
            if not color_applied and lease_rate is not None:
                if lease_rate >= 10:  # 전세증감율 10% 이상 상승
                    tags = ['positive_lease']
                    color_applied = True
                elif lease_rate <= -10:  # 전세증감율 10% 이상 하락
                    tags = ['negative_lease']
                    color_applied = True
            
            # 4순위: 일반적인 매매증감율 색상
            if not color_applied and sale_rate is not None:
                if sale_rate > 0:
                    tags = ['positive_sale']
                elif sale_rate < 0:
                    tags = ['negative_sale']
                else:
                    tags = [base_tag]  # 0%는 기본 색상
            
            # 색상이 적용되지 않은 경우 기본 zebra striping 적용
            if not tags:
                tags = [base_tag]
            
            # Treeview에 항목 삽입
            self.tree.insert('', tk.END, values=values, iid=str(item.get('면적일련번호', '')), tags=tags)

        self.loading_label.config(text=f"총 {len(self.filtered_data)}개의 결과를 조회했습니다.")
        if not self.filtered_data:
            self.loading_label.config(text="조회 조건에 맞는 데이터가 없습니다.")

    def filter_data(self, data):
        """조회된 데이터 필터링"""
        filtered = []
        for item in data:
            try:
                area_str = str(item.get('전용면적', '0')).strip()
                if area_str and area_str != '':
                    area = float(area_str)
                    if area < float(self.exclusive_area_min.get()) or area > float(self.exclusive_area_max.get()):
                        continue

                years_str = str(item.get('연차', '0')).strip()
                if years_str and years_str != '':
                    years = int(float(years_str))
                    if years > int(self.max_years.get()):
                        continue

                unit_price_str = str(item.get('평단가', '0')).strip()
                if unit_price_str and unit_price_str != '':
                    unit_price = float(unit_price_str)
                    if unit_price > float(self.max_unit_price.get()):
                        continue

                sale_rate_str = str(item.get('매매증감율', '0')).strip()
                if sale_rate_str and sale_rate_str != '':
                    sale_rate = float(sale_rate_str)
                    if sale_rate < float(self.rate_min.get()) or sale_rate > float(self.rate_max.get()):
                        continue
                filtered.append(item)
            except (ValueError, TypeError) as e:
                print(f"필터링 중 오류 발생: {e}, 데이터: {item}")
                filtered.append(item)
                continue
        return filtered

    def export_to_excel(self):
        """Excel로 데이터 내보내기"""
        if not hasattr(self, 'filtered_data') or not self.filtered_data:
            messagebox.showwarning("경고", "내보낼 데이터가 없습니다. 먼저 조회를 실행하세요.")
            return
        try:
            df = pd.DataFrame(self.filtered_data)
            columns_order = ['지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율',
                             '매매시세', '전세시세', '전세가율', '매전갭', '세대수', '면적일련번호', '단지기본일련번호']
            df = df.reindex(columns=columns_order, fill_value='')

            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"투자테이블_{timestamp}.xlsx"

            save_directory = self.export_path_var.get()
            if not save_directory:
                self.set_default_export_path()
                save_directory = self.export_path_var.get()
            
            full_path = os.path.join(save_directory, filename)

            df.to_excel(full_path, index=False, engine='openpyxl')

            messagebox.showinfo("성공", f"데이터가 '{full_path}' 파일로 내보내기 완료되었습니다.")
        except Exception as e:
            messagebox.showerror("오류", f"Excel 내보내기 중 오류가 발생했습니다: {str(e)}")

    def create_result_table(self, parent):
        """투자테이블 결과 Treeview 생성"""
        table_frame = ttk.Frame(parent)
        table_frame.pack(fill=tk.BOTH, expand=True)

        columns = ('지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율', '매매시세', '전세시세', '전세가율', '매전갭', '세대수')
        self.tree = ttk.Treeview(table_frame, columns=columns, show='tree headings', height=20)

        # === 색상 태그 설정 ===
        # 매매증감율 관련 색상
        self.tree.tag_configure('positive_sale', background='#E8F5E8', foreground='#2E7D32')  # 연한 녹색 배경, 진한 녹색 글자
        self.tree.tag_configure('negative_sale', background='#FFEBEE', foreground='#C62828')  # 연한 빨간색 배경, 진한 빨간색 글자
        
        # 전세증감율 관련 색상
        self.tree.tag_configure('positive_lease', background='#E3F2FD', foreground='#1565C0')  # 연한 파란색 배경, 진한 파란색 글자
        self.tree.tag_configure('negative_lease', background='#FFF3E0', foreground='#E65100')  # 연한 주황색 배경, 진한 주황색 글자
        
        # 전세가율 관련 색상
        self.tree.tag_configure('high_ratio', background='#FCE4EC', foreground='#AD1457')     # 높은 전세가율 (90% 이상)
        
        # 기본 zebra striping
        self.tree.tag_configure('oddrow', background='#F8F8F8')
        self.tree.tag_configure('evenrow', background='#FFFFFF')
        
        # 중성 색상
        self.tree.tag_configure('neutral', background='#F5F5F5', foreground='#424242')

        self.tree.column("#0", width=30, minwidth=30, stretch=tk.NO)
        self.tree.heading("#0", text="", anchor=tk.CENTER)

        column_widths = {
            '지역': 120, '단지명': 150, '전용면적': 80, '연차': 60, '평단가': 100,
            '매매증감율': 100, '전세증감율': 100, '매매시세': 100, '전세시세': 100,
            '전세가율': 80, '매전갭': 80, '세대수': 80
        }

#        self.sort_directions = {}

        # === 컬럼 헤더 스타일링 ===
        style = ttk.Style()
        
        # 기본 헤더 스타일
        style.configure("Treeview.Heading", 
                       background="#2196F3",   # 파란색 배경
                       foreground="black",     # 흰색 글자
                       font=("Arial", 10, "bold"))
        
        # 마우스 오버 효과
        style.map("Treeview.Heading",
                  background=[('active', '#1976D2')])  # 마우스 오버시 더 진한 파란색

#        for col in columns:
#            self.tree.heading(col, text=col, command=lambda _col=col: self._sort_column(_col))
#            self.tree.column(col, width=column_widths.get(col, 100), anchor=tk.CENTER)
#            self.sort_directions[col] = False
        # 🟢 수정된 부분
        for col in columns:
            # 투자테이블 전용 정렬 메서드 사용
            self.tree.heading(col, text=col, command=lambda _col=col: self._sort_investment_table_column(_col))
            self.tree.column(col, width=column_widths.get(col, 100), anchor=tk.CENTER)
            # 투자테이블 전용 정렬 딕셔너리에 초기값 설정
            self.investment_table_sort_directions[col] = False
#        for col in columns:
#            # 투자테이블 전용 정렬 메서드 사용
#            self.tree.heading(col, text=col, command=lambda _col=col: self._sort_investment_table_column(_col))
#            self.tree.column(col, width=column_widths.get(col, 100), anchor=tk.CENTER)
#            self.investment_table_sort_directions[col] = False  # 투자테이블 전용 정렬 딕셔너리 사용

        scrollbar_y = ttk.Scrollbar(table_frame, orient=tk.VERTICAL, command=self.tree.yview)
        scrollbar_x = ttk.Scrollbar(table_frame, orient=tk.HORIZONTAL, command=self.tree.xview)

        self.tree.configure(yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set)

        self.tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
        scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)

        self.tree.bind("<<TreeviewSelect>>", self._on_tree_select)

    def _sort_investment_table_column(self, col):
        """투자테이블 전용 컬럼 정렬 메서드"""
        if not self.filtered_data:
            return
        
        # 현재 정렬 방향 확인 및 토글
        reverse = self.investment_table_sort_directions.get(col, False)
        self.investment_table_sort_directions[col] = not reverse
        
        # 🟢 추가: 모든 헤더에서 정렬 기호 제거 후 현재 컬럼에만 표시
        columns = ('지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율', '매매시세', '전세시세', '전세가율', '매전갭', '세대수')
        
        for header_col in columns:
            if header_col == col:
                # 현재 정렬된 컬럼에 화살표 추가
                arrow = " ▼" if not reverse else " ▲"  # reverse가 False면 내림차순(▼), True면 오름차순(▲)
                self.tree.heading(header_col, text=f"{header_col}{arrow}")
            else:
                # 다른 컬럼들은 화살표 제거
                self.tree.heading(header_col, text=header_col)
        
        # 데이터 정렬
        try:
            # 숫자형 컬럼들을 위한 정렬 키 함수
            def sort_key(item):
                value = item.get(col, '')
                if col in ['전용면적', '연차', '평단가', '매매증감율', '전세증감율', 
                          '매매시세', '전세시세', '전세가율', '매전갭', '세대수']:
                    try:
                        # 숫자 변환 시도
                        return float(str(value).replace(',', '').strip() or '0')
                    except (ValueError, TypeError):
                        return 0
                else:
                    # 문자열 컬럼은 그대로 반환
                    return str(value)
            
            # 정렬된 데이터로 업데이트
            self.filtered_data.sort(key=sort_key, reverse=reverse)
            
            # Treeview 업데이트
            for item in self.tree.get_children():
                self.tree.delete(item)
            
            # 정렬된 데이터를 다시 삽입 (기존 로직과 동일)
            for i, item in enumerate(self.filtered_data):
                values = (
                    item.get('지역', ''), item.get('단지명', ''), item.get('전용면적', ''),
                    item.get('연차', ''), item.get('평단가', ''), item.get('매매증감율', ''),
                    item.get('전세증감율', ''), item.get('매매시세', ''), item.get('전세시세', ''),
                    item.get('전세가율', ''), item.get('매전갭', ''), item.get('세대수', '')
                )
                
                # 색상 태그 다시 적용 (기존 색상 로직과 동일)
                tags = []
                base_tag = 'evenrow' if i % 2 == 0 else 'oddrow'
                
                try:
                    sale_rate = None
                    lease_rate = None
                    lease_ratio = None
                    
                    # 매매증감율 파싱
                    sale_rate_str = str(item.get('매매증감율', '0')).strip()
                    if sale_rate_str and sale_rate_str not in ['', '-', '0']:
                        sale_rate = float(sale_rate_str)
                    
                    # 전세증감율 파싱
                    lease_rate_str = str(item.get('전세증감율', '0')).strip()
                    if lease_rate_str and lease_rate_str not in ['', '-', '0']:
                        lease_rate = float(lease_rate_str)
                    
                    # 전세가율 파싱
                    lease_ratio_str = str(item.get('전세가율', '0')).strip()
                    if lease_ratio_str and lease_ratio_str not in ['', '-', '0']:
                        lease_ratio = float(lease_ratio_str)
                        
                except (ValueError, TypeError):
                    pass
                
                # 색상 우선순위 적용 (기존과 동일한 로직)
                color_applied = False
                
                if lease_ratio is not None and lease_ratio >= 90:
                    tags = ['high_ratio']
                    color_applied = True
                elif sale_rate is not None:
                    if sale_rate >= 10:
                        tags = ['positive_sale']
                        color_applied = True
                    elif sale_rate <= -10:
                        tags = ['negative_sale']
                        color_applied = True
                
                if not color_applied and lease_rate is not None:
                    if lease_rate >= 10:
                        tags = ['positive_lease']
                        color_applied = True
                    elif lease_rate <= -10:
                        tags = ['negative_lease']
                        color_applied = True
                
                if not color_applied and sale_rate is not None:
                    if sale_rate > 0:
                        tags = ['positive_sale']
                    elif sale_rate < 0:
                        tags = ['negative_sale']
                    else:
                        tags = [base_tag]
                
                if not tags:
                    tags = [base_tag]
                
                self.tree.insert('', tk.END, values=values, 
                               iid=str(item.get('면적일련번호', '')), tags=tags)
                               
        except Exception as e:
            print(f"정렬 중 오류 발생: {e}")
            messagebox.showerror("정렬 오류", f"데이터 정렬 중 오류가 발생했습니다: {e}")

    def _sort_column(self, tree_widget, sort_directions_dict, col): # tree_widget과 sort_directions_dict 인자 추가
        """범용 Treeview 컬럼 정렬 (투자테이블, 매물수집기 등에서 사용 가능)"""
        current_data = []
        for item_id in tree_widget.get_children(): # self.tree 대신 tree_widget 사용
            current_data.append((tree_widget.set(item_id, col), item_id)) # self.tree 대신 tree_widget 사용

        def get_sort_key(item_tuple):
            value = item_tuple[0]
            try:
                # 매물수집기 Treeview 컬럼 추가 ('atclNo', 'prc', 'rentPrc', 'spc1', 'spc2')
                # 숫자로 변환될 수 있는 컬럼들을 확장합니다.
                numeric_cols = [
                    '전용면적', '연차', '평단가', '매매증감율', '전세증감율', '매매시세', '전세시세', '전세가율', '매전갭', '세대수', # 투자테이블용
                    "atclNo", "prc", "rentPrc", "spc1", "spc2" # 매물수집기용 (새로 추가)
                ]
                if col in numeric_cols:
                    # 'prc'와 'rentPrc'가 "만원" 또는 "/"를 포함할 수 있으므로 숫자로 정제
                    if isinstance(value, str):
                        value = value.replace(' 만원', '').replace(',', '')
                        if '/' in value: # 월세 (보증금/월세) 처리
                            parts = value.split('/')
                            # 보증금 + 월세 (월세를 더 중요하게 정렬하고 싶다면 월세 부분만 사용)
                            value = float(parts[0]) + float(parts[1]) if len(parts) == 2 else 0.0
                        else:
                            value = float(value)
                    return float(value) if value is not None and value != '' else -float('inf')
                else:
                    return value
            except (ValueError, TypeError): # 숫자 변환 실패 시 문자열로 처리
                return str(value) # 숫자로 변환되지 않으면 문자열로 비교

        # self.sort_directions 대신 인자로 받은 sort_directions_dict 사용
        reverse_sort = not sort_directions_dict.get(col, False) # 기본값 False 추가
        sort_directions_dict[col] = reverse_sort
        current_data.sort(key=get_sort_key, reverse=reverse_sort)

        for index, (value, item_id) in enumerate(current_data):
            tree_widget.move(item_id, '', index) # self.tree 대신 tree_widget 사용

        # 모든 컬럼 헤더의 정렬 표시 업데이트
        # Treeview마다 컬럼 정의가 다르므로 tree_widget['columns']를 사용
        for c in tree_widget['columns']:
            if c == col:
                tree_widget.heading(c, text=f"{tree_widget.heading(c, 'text').split(' ')[0]} {'▼' if reverse_sort else '▲'}")
            else:
                # 정렬 표시를 제거하려면 헤더 텍스트에서 ▲/▼ 제거 로직 필요
                original_text = tree_widget.heading(c, 'text').split(' ')[0]
                tree_widget.heading(c, text=original_text)

        # 현재 정렬 방향을 저장하기 위한 딕셔너리를 InvestmentTableProgram 클래스에 추가해야 합니다.
        # self.collector_sort_directions = {} 이런 식으로요.

    def _on_tree_select(self, event):
        """투자테이블 Treeview 항목 선택 이벤트 핸들러"""
        selected_items_ids = self.tree.selection()
        temp_selected_apartments = {}
        for item_id in selected_items_ids:
            original_item_found = None
            for original_parsed_item in self.filtered_data:
                if str(original_parsed_item.get('면적일련번호')) == item_id:
                    original_item_found = original_parsed_item
                    break
            if original_item_found:
                temp_selected_apartments[item_id] = {
                    '지역': original_parsed_item.get('지역', ''),
                    '단지명': original_parsed_item.get('단지명', ''),
                    '전용면적': original_parsed_item.get('전용면적', ''),
                    '연차': original_parsed_item.get('연차', ''),
                    '면적일련번호': original_parsed_item.get('면적일련번호', ''),
                    '단지기본일련번호': original_parsed_item.get('단지기본일련번호', '')
                }
        self.selected_apartments = temp_selected_apartments
        if selected_items_ids:
            self.current_selected_apt_info = self.selected_apartments.get(selected_items_ids[-1])
        else:
            self.current_selected_apt_info = None
        pass

    def create_chart_page(self, parent_frame):
        """2페이지: 시세차트 UI 생성 (매물 검색 기능 제거됨)"""
        pane_window = ttk.PanedWindow(parent_frame, orient=tk.HORIZONTAL)
        pane_window.pack(fill=tk.BOTH, expand=True)
        
        left_frame = ttk.Frame(pane_window, width=200, relief=tk.SUNKEN)
        pane_window.add(left_frame, weight=1)

        ttk.Label(left_frame, text="선택 단지 목록", font=('Arial', 14, 'bold')).pack(pady=10)

        selected_apt_columns = ('지역', '단지명', '전용면적', '연차')
        self.selected_apt_tree = ttk.Treeview(left_frame, columns=selected_apt_columns, show='headings', height=15)

        for col in selected_apt_columns:
            self.selected_apt_tree.heading(col, text=col, anchor=tk.CENTER)
            self.selected_apt_tree.column(col, width=80, anchor=tk.CENTER)

        selected_apt_scrollbar_y = ttk.Scrollbar(left_frame, orient=tk.VERTICAL, command=self.selected_apt_tree.yview)
        self.selected_apt_tree.configure(yscrollcommand=selected_apt_scrollbar_y.set)

        self.selected_apt_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        selected_apt_scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)

        self.selected_apt_tree.bind("<<TreeviewSelect>>", self.on_selected_apt_tree_select)

        # 중앙 프레임
        center_frame = ttk.Frame(pane_window, relief=tk.SUNKEN)
        pane_window.add(center_frame, weight=6)

        center_frame.grid_rowconfigure(0, weight=0)
        center_frame.grid_rowconfigure(1, weight=6)
        center_frame.grid_rowconfigure(2, weight=2)
        center_frame.grid_columnconfigure(0, weight=1)

        # 차트 옵션창
        chart_options_frame = ttk.LabelFrame(center_frame, text="차트 옵션", padding=10)
        chart_options_frame.grid(row=0, column=0, sticky="ew", padx=5, pady=5)

        # 차트 기간 설정 UI
        ttk.Label(chart_options_frame, text="차트 시작일:").grid(row=0, column=0, padx=5, pady=2, sticky=tk.W)
        ttk.Entry(chart_options_frame, textvariable=self.chart_start_date_var, width=15).grid(row=0, column=1, padx=5, pady=2, sticky=tk.W)
        ttk.Label(chart_options_frame, text="차트 종료일:").grid(row=0, column=2, padx=5, pady=2, sticky=tk.W)
        ttk.Entry(chart_options_frame, textvariable=self.chart_end_date_var, width=15).grid(row=0, column=3, padx=5, pady=2, sticky=tk.W)
        ttk.Button(chart_options_frame, text="기간 적용", command=self.update_chart_view).grid(row=0, column=4, padx=10, pady=2)

        # 차트 표시 유형 선택 체크버튼
        ttk.Checkbutton(chart_options_frame, text="매매가", variable=self.show_sise_sale_var, command=self.update_chart_view).grid(row=1, column=0, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세가", variable=self.show_sise_lease_var, command=self.update_chart_view).grid(row=1, column=1, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세가율", variable=self.show_sise_ratio_var, command=self.update_chart_view).grid(row=1, column=2, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="매매실거래", variable=self.show_real_sale_var, command=self.update_chart_view).grid(row=1, column=3, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세실거래", variable=self.show_real_lease_var, command=self.update_chart_view).grid(row=1, column=4, padx=5, pady=2, sticky=tk.W)

        # 시세차트 영역
        self.chart_frame = ttk.Frame(center_frame, relief=tk.GROOVE, borderwidth=2)
        self.chart_frame.grid(row=1, column=0, sticky="nsew", padx=5, pady=5)

        self.fig = Figure(figsize=(6, 6), dpi=100)
        gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
        self.ax = self.fig.add_subplot(gs[0, 0])
        self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.chart_frame)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # 차트 하단 주요 가격 정보
        self.chart_info_frame = ttk.Frame(center_frame, relief=tk.GROOVE, borderwidth=2)
        self.chart_info_frame.grid(row=2, column=0, sticky="nsew", padx=5, pady=5)

        # 3분할 프레임 및 레이블 생성
        price_info_frame1 = ttk.Frame(self.chart_info_frame)
        price_info_frame1.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)
        price_info_frame2 = ttk.Frame(self.chart_info_frame)
        price_info_frame2.grid(row=0, column=1, sticky="nsew", padx=5, pady=5)
        price_info_frame3 = ttk.Frame(self.chart_info_frame)
        price_info_frame3.grid(row=0, column=2, sticky="nsew", padx=5, pady=5)
        
        self.chart_info_frame.grid_columnconfigure(0, weight=1)
        self.chart_info_frame.grid_columnconfigure(1, weight=1)
        self.chart_info_frame.grid_columnconfigure(2, weight=1)

        # 좌측 섹션: 매매 최고/최근가, 전세 최고/최근가
        ttk.Label(price_info_frame1, text="<시세 정보>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels = {}
        self.chart_info_labels['매매최고가'] = ttk.Label(price_info_frame1, text="매매최고가: -")
        self.chart_info_labels['매매최고가'].pack(anchor=tk.W)
        self.chart_info_labels['매매최근가'] = ttk.Label(price_info_frame1, text="매매최근가: -")
        self.chart_info_labels['매매최근가'].pack(anchor=tk.W)
        self.chart_info_labels['전세최고가'] = ttk.Label(price_info_frame1, text="전세최고가: -")
        self.chart_info_labels['전세최고가'].pack(anchor=tk.W)
        self.chart_info_labels['전세최근가'] = ttk.Label(price_info_frame1, text="전세최근가: -")
        self.chart_info_labels['전세최근가'].pack(anchor=tk.W)

        # 가운데 섹션: 실거래 매매/전세 최고/최근가
        ttk.Label(price_info_frame2, text="<실거래 정보>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['실거래매매최고가'] = ttk.Label(price_info_frame2, text="실거래매매최고가: -")
        self.chart_info_labels['실거래매매최고가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래매매최근가'] = ttk.Label(price_info_frame2, text="실거래매매최근가: -")
        self.chart_info_labels['실거래매매최근가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래전세최고가'] = ttk.Label(price_info_frame2, text="실거래전세최고가: -")
        self.chart_info_labels['실거래전세최고가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래전세최근가'] = ttk.Label(price_info_frame2, text="실거래전세최근가: -")
        self.chart_info_labels['실거래전세최근가'].pack(anchor=tk.W)

        # 우측 섹션: 매매 하락가/율, 회복율
        ttk.Label(price_info_frame3, text="<매매 분석>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['매매하락가'] = ttk.Label(price_info_frame3, text="매매하락가: -")
        self.chart_info_labels['매매하락가'].pack(anchor=tk.W)
        self.chart_info_labels['매매하락율'] = ttk.Label(price_info_frame3, text="매매하락율: -")
        self.chart_info_labels['매매하락율'].pack(anchor=tk.W)
        self.chart_info_labels['매매회복율'] = ttk.Label(price_info_frame3, text="매매회복율: -")
        self.chart_info_labels['매매회복율'].pack(anchor=tk.W)

        # 우측 프레임 (실거래 정보만 포함)
        right_frame = ttk.PanedWindow(pane_window, orient=tk.VERTICAL)
        pane_window.add(right_frame, weight=1)

        # 우측 상단 - 실거래 정보창 (좌우 2분할)
        top_right_frame = ttk.Frame(right_frame, relief=tk.GROOVE, borderwidth=2)
        right_frame.add(top_right_frame, weight=1)

        top_right_frame.grid_columnconfigure(0, weight=1)
        top_right_frame.grid_columnconfigure(1, weight=1)
        top_right_frame.grid_rowconfigure(0, weight=0)
        top_right_frame.grid_rowconfigure(1, weight=1)

        # 매매 실거래 Treeview (좌측 컬럼)
        sale_label_frame = ttk.Frame(top_right_frame)
        sale_label_frame.grid(row=0, column=0, sticky="ew", padx=2, pady=2)
        ttk.Label(sale_label_frame, text="매매 실거래", font=('Arial', 10, 'bold')).pack(pady=(5, 0))

        sale_tree_frame = ttk.Frame(top_right_frame)
        sale_tree_frame.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        sale_real_tran_cols = ('날짜', '실거래금액', '층수')
        self.sale_real_tran_tree = ttk.Treeview(sale_tree_frame, columns=sale_real_tran_cols, show='headings')
        for col in sale_real_tran_cols:
            self.sale_real_tran_tree.heading(col, text=col, anchor=tk.CENTER)
            self.sale_real_tran_tree.column(col, width=70, anchor=tk.CENTER)
        self.sale_real_tran_tree.pack(fill=tk.BOTH, expand=True)

        # 전세/월세 실거래 Treeview (우측 컬럼)
        lease_label_frame = ttk.Frame(top_right_frame)
        lease_label_frame.grid(row=0, column=1, sticky="ew", padx=2, pady=2)
        ttk.Label(lease_label_frame, text="전세/월세 실거래", font=('Arial', 10, 'bold')).pack(pady=(5, 0))

        lease_tree_frame = ttk.Frame(top_right_frame)
        lease_tree_frame.grid(row=1, column=1, sticky="nsew", padx=2, pady=2)
        lease_real_tran_cols = ('날짜', '실거래금액', '층수')
        self.lease_real_tran_tree = ttk.Treeview(lease_tree_frame, columns=lease_real_tran_cols, show='headings')
        for col in lease_real_tran_cols:
            self.lease_real_tran_tree.heading(col, text=col, anchor=tk.CENTER)
            self.lease_real_tran_tree.column(col, width=70, anchor=tk.CENTER)
        self.lease_real_tran_tree.pack(fill=tk.BOTH, expand=True)

    def _update_selected_apt_tree(self):
        """시세차트 페이지의 선택 단지 목록 Treeview를 업데이트합니다."""
        # Treeview의 기존 내용을 모두 삭제
        for item in self.selected_apt_tree.get_children():
            self.selected_apt_tree.delete(item)

        # self.selected_apartments 딕셔너리의 최신 내용으로 Treeview를 다시 채움
        # self.selected_apartments 딕셔너리는 _on_tree_select 메서드에서 관리됩니다.
        for item_id, apt_info in self.selected_apartments.items():
            self.selected_apt_tree.insert('', tk.END, values=(
                apt_info.get('지역', ''),
                apt_info.get('단지명', ''),
                apt_info.get('전용면적', ''),
                apt_info.get('연차', '')
            ), iid=item_id) # 'iid'에 item_id를 사용하여 Treeview 항목을 식별합니다.

        # UI 하단 상태바에 업데이트 메시지 표시 (옵션)
        self.update_status(f"시세차트: {len(self.selected_apartments)}개 단지 목록 업데이트 완료.")

    def on_selected_apt_tree_select(self, event):
        """시세차트 선택 단지 Treeview 선택 이벤트 핸들러"""
        selected_item_id = self.selected_apt_tree.focus()
        if selected_item_id and selected_item_id in self.selected_apartments:
            apt_info = self.selected_apartments[selected_item_id]
            complex_serial_no = apt_info.get('단지기본일련번호')
            area_serial_no = apt_info.get('면적일련번호')
            self.current_selected_apt_info = apt_info
            if complex_serial_no and area_serial_no:
                self.loading_label.config(text=f"'{apt_info.get('단지명')} {apt_info.get('전용면적')}㎡' 시세 데이터를 조회 중입니다...")
                thread_chart = threading.Thread(target=self.fetch_chart_data, args=(complex_serial_no, area_serial_no))
                thread_chart.daemon = True
                thread_chart.start()
                thread_real_tran = threading.Thread(target=self.fetch_real_transaction_data, args=(complex_serial_no, area_serial_no))
                thread_real_tran.daemon = True
                thread_real_tran.start()
            else:
                messagebox.showwarning("경고", "선택된 단지의 단지기본일련번호 또는 면적일련번호가 없습니다.")
                self.loading_label.config(text="데이터 조회 실패: 필수 정보 누락")
        else:
            self.loading_label.config(text="단지를 선택해주세요.")
            self.current_selected_apt_info = None

    def fetch_chart_data(self, complex_serial_no, area_serial_no):
        """KB 시세 차트 데이터 가져오기 (update_chart_display 호출)"""
        try:
            end_date = datetime.now()
            start_date_api = "19700101"
            end_date_api = end_date.strftime("%Y%m%d")

            base_url = "https://api.kbland.kr/land-price/price/PerMn/IntgrationChart"
            params = {
                "단지기본일련번호": complex_serial_no, "면적일련번호": area_serial_no,
                "거래구분": "0", "조회구분": "2",
                "조회시작일": start_date_api, "조회종료일": end_date_api
            }
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            headers = {
                'User-Agent': 'A',
                #'Accept': 'application/json, text/plain, */*',
                #'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                #'Accept-Encoding': 'gzip, deflate, br',
                #'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = session.get(url, timeout=30)
                    response.raise_for_status()
                    break
                except requests.exceptions.ConnectionError as e:
                    if attempt == max_retries - 1:
                        raise e
                    time.sleep(2)

            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                raise ValueError(f"JSON 응답이 아닙니다. Content-Type: {response.headers.get('content-type')}")

            self.root.after(0, self.update_chart_display, data)

        except Exception as e:
            error_msg = f"Chart API 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)

    def fetch_real_transaction_data(self, complex_serial_no, area_serial_no):
        """KB 실거래 정보 데이터 가져오기 (parse_real_transaction_data 호출)"""
        try:
            base_url = "https://api.kbland.kr/land-price/price/LatestRealTranPrc"
            params = {
                "단지기본일련번호": complex_serial_no,
                "면적일련번호": area_serial_no,
                "거래구분": "0",  # 0: 전체
                "조회구분": "0",  # 0: 전체
                "첫페이지갯수": "100000",  # 충분히 큰 값으로 모든 데이터 가져오기 시도
                "현재페이지": "1",
                "페이지갯수": "0"
            }
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            headers = {
                'User-Agent': 'A',
                #'Accept': 'application/json, text/plain, */*',
                #'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                #'Accept-Encoding': 'gzip, deflate, br',
                #'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = session.get(url, timeout=30)
                    response.raise_for_status()
                    break
                except requests.exceptions.ConnectionError as e:
                    if attempt == max_retries - 1:
                        raise e
                    time.sleep(2)

            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                raise ValueError("실거래 API JSON 응답이 아닙니다.")
            
            sale_list, lease_list = self.parse_real_transaction_data(data)
            self.root.after(0, self.display_real_transaction_data, sale_list, lease_list)

        except Exception as e:
            self.root.after(0, self.show_error, f"실거래 API 오류: {e}")

    def parse_real_transaction_data(self, raw_data):
        """KB 실거래 정보 API 응답 데이터 파싱 및 분류"""
        sale_list = []
        lease_list = []
        items = raw_data.get('dataBody', {}).get('data', []) 
        
        for item in items:
            물건거래명 = item.get('물건거래명', '')
            계약년월 = item.get('계약년월', '')
            계약일 = item.get('계약일', '')
            해당층수 = item.get('해당층수', '-')
            
            날짜 = f"{계약년월[:4]}-{계약년월[4:]}-{계약일}" if 계약년월 and 계약일 else '-'
            
            거래금액_표시 = '-'
            if 물건거래명 == '매매':
                금액_str = str(item.get('매매실거래금액', '0')).replace(',', '')
                거래금액_표시 = f"{float(금액_str):.0f} 만원" if 금액_str and float(금액_str) > 0 else '-'
                sale_list.append((날짜, 거래금액_표시, 해당층수))
            elif 물건거래명 == '전세':
                금액_str = str(item.get('전세실거래금액', '0')).replace(',', '')
                거래금액_표시 = f"{float(금액_str):.0f} 만원" if 금액_str and float(금액_str) > 0 else '-'
                lease_list.append((날짜, 거래금액_표시, 해당층수))
            elif 물건거래명 == '월세':
                보증금액_str = str(item.get('보증금액', '0')).replace(',', '')
                월세금액_str = str(item.get('월세금액', '0')).replace(',', '')
                보증금 = f"{float(보증금액_str):.0f}" if 보증금액_str and float(보증금액_str) > 0 else '0'
                월세 = f"{float(월세금액_str):.0f}" if 월세금액_str and float(월세금액_str) > 0 else '0'
                거래금액_표시 = f"{보증금}/{월세} 만원"
                lease_list.append((날짜, 거래금액_표시, 해당층수))
            
        sale_list.sort(key=lambda x: x[0], reverse=True)
        lease_list.sort(key=lambda x: x[0], reverse=True)

        return sale_list, lease_list

    def display_real_transaction_data(self, sale_list, lease_list):
        """실거래 정보를 Treeview에 표시"""
        # 기존 데이터 삭제
        for item in self.sale_real_tran_tree.get_children():
            self.sale_real_tran_tree.delete(item)
        for item in self.lease_real_tran_tree.get_children():
            self.lease_real_tran_tree.delete(item)
        
        # 새 데이터 삽입
        for row_data in sale_list:
            self.sale_real_tran_tree.insert('', tk.END, values=row_data)
        for row_data in lease_list:
            self.lease_real_tran_tree.insert('', tk.END, values=row_data)

    def update_chart_display(self, chart_data_raw):
        """시세 차트 화면 업데이트 (update_chart_view 호출)"""
        parsed_chart_data = self.parse_chart_api_data(chart_data_raw)
        self.current_chart_data = parsed_chart_data

        if parsed_chart_data and (parsed_chart_data['sise_data'] or \
           parsed_chart_data['sale_real_data'] or parsed_chart_data['lease_real_data']):

            all_dates_in_chart_data = []
            if parsed_chart_data['sise_data']:
                all_dates_in_chart_data.extend([pd.to_datetime(d['기준년월'], format='%Y%m') for d in parsed_chart_data['sise_data']])
            if parsed_chart_data['sale_real_data']:
                all_dates_in_chart_data.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in parsed_chart_data['sale_real_data'] if d['거래금액'] is not None])
            if parsed_chart_data['lease_real_data']:
                all_dates_in_chart_data.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in parsed_chart_data['lease_real_data'] if d['거래금액'] is not None])

            if all_dates_in_chart_data:
                min_date = min(all_dates_in_chart_data)
                max_date = max(all_dates_in_chart_data)
                self.chart_start_date_var.set(min_date.strftime("%Y-%m-%d"))
                self.chart_end_date_var.set(max_date.strftime("%Y-%m-%d"))
            else:
                self.chart_start_date_var.set("")
                self.chart_end_date_var.set("")

            self.update_chart_view()
            self.loading_label.config(text="시세 차트가 업데이트되었습니다.")
        else:
            self.fig.clf()
            gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
            self.ax = self.fig.add_subplot(gs[0, 0])
            self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)
            self.ax.text(0.5, 0.5, "데이터가 없습니다.", ha='center', va='center', transform=self.ax.transAxes)
            self.canvas.draw()
            self.update_chart_summary_info(None)
            self.loading_label.config(text="선택된 단지의 시세 데이터를 찾을 수 없습니다.")
            self.chart_start_date_var.set("")
            self.chart_end_date_var.set("")

    def update_chart_view(self):
        """차트 옵션 및 기간에 따라 차트 뷰 업데이트 (draw_price_chart, update_chart_summary_info 호출)"""
        if not self.current_chart_data:
            self.loading_label.config(text="차트 데이터가 없습니다.")
            return

        filtered_sise_data = []
        filtered_sale_real_data = []
        filtered_lease_real_data = []

        try:
            chart_start_date = datetime.strptime(self.chart_start_date_var.get(), "%Y-%m-%d").date()
            chart_end_date = datetime.strptime(self.chart_end_date_var.get(), "%Y-%m-%d").date()
            if chart_start_date > chart_end_date:
                messagebox.showwarning("경고", "차트 시작일은 종료일보다 빨라야 합니다.")
                return
        except ValueError:
            messagebox.showwarning("경고", "차트 기간 형식이 올바르지 않습니다 (YYYY-MM-DD).")
            return

        for item in self.current_chart_data['sise_data']:
            item_date = datetime.strptime(item['기준년월'], "%Y%m").date()
            if chart_start_date <= item_date <= chart_end_date:
                filtered_sise_data.append(item)

        for item in self.current_chart_data['sale_real_data']:
            item_date = datetime.strptime(item['기준년월일'], "%Y%m%d").date()
            if chart_start_date <= item_date <= chart_end_date:
                filtered_sale_real_data.append(item)

        for item in self.current_chart_data['lease_real_data']:
            item_date = datetime.strptime(item['기준년월일'], "%Y%m%d").date()
            if chart_start_date <= item_date <= chart_end_date:
                filtered_lease_real_data.append(item)

        self.draw_price_chart(
            {'sise_data': filtered_sise_data, 'sale_real_data': filtered_sale_real_data, 'lease_real_data': filtered_lease_real_data},
            show_sise_sale=self.show_sise_sale_var.get(),
            show_sise_lease=self.show_sise_lease_var.get(),
            show_sise_ratio=self.show_sise_ratio_var.get(),
            show_real_sale=self.show_real_sale_var.get(),
            show_real_lease=self.show_real_lease_var.get(),
            apt_info=self.current_selected_apt_info  # 단지 정보 전달
        )
        self.update_chart_summary_info(
            {'sise_data': filtered_sise_data, 'sale_real_data': filtered_sale_real_data, 'lease_real_data': filtered_lease_real_data}
        )

    def parse_chart_api_data(self, chart_data_raw):
        """시세 차트 API 응답 데이터 파싱"""
        sise_data = []
        sale_real_data = []
        lease_real_data = []
        try:
            data_body_data = chart_data_raw.get('dataBody', {}).get('data', {})

            sise_groups = data_body_data.get('시세', [])
            for group in sise_groups:
                if 'items' in group and isinstance(group['items'], list):
                    for item in group['items']:
                        sise_data.append({
                            '기준년월': item.get('기준년월', ''),
                            '매매일반거래가': item.get('매매일반거래가', None),
                            '전세일반거래가': item.get('전세일반거래가', None),
                            '전세가율': item.get('전세가율', None)
                        })
                        if '매매실거래금액리스트' in item and isinstance(item['매매실거래금액리스트'], list):
                            기준년월 = item.get('기준년월', '')
                            for 금액_str in item['매매실거래금액리스트']:
                                거래금액 = float(str(금액_str).replace(',', '')) if str(금액_str).replace(',', '') else None
                                if 거래금액 is not None:
                                    sale_real_data.append({'기준년월일': 기준년월 + '01', '거래금액': 거래금액})
                        if '전세실거래금액리스트' in item and isinstance(item['전세실거래금액리스트'], list):
                            기준년월 = item.get('기준년월', '')
                            for 금액_str in item['전세실거래금액리스트']:
                                거래금액 = float(str(금액_str).replace(',', '')) if str(금액_str).replace(',', '') else None
                                if 거래금액 is not None:
                                    lease_real_data.append({'기준년월일': 기준년월 + '01', '거래금액': 거래금액})
            sise_data.sort(key=lambda x: x['기준년월'])

            sale_real_df = pd.DataFrame(sale_real_data)
            if not sale_real_df.empty:
                sale_real_data = sale_real_df.drop_duplicates(subset=['기준년월일', '거래금액']).to_dict(orient='records')
            lease_real_df = pd.DataFrame(lease_real_data)
            if not lease_real_df.empty:
                lease_real_data = lease_real_df.drop_duplicates(subset=['기준년월일', '거래금액']).to_dict(orient='records')

            sale_real_data.sort(key=lambda x: x['기준년월일'])
            lease_real_data.sort(key=lambda x: x['기준년월일'])

            return {
                'sise_data': sise_data,
                'sale_real_data': sale_real_data,
                'lease_real_data': lease_real_data
            }
        except Exception as e:
            self.root.after(0, self.show_error, f"차트 데이터 파싱 오류: {e}")
            return None

    def draw_price_chart(self, chart_data, show_sise_sale=True, show_sise_lease=True, show_sise_ratio=True, show_real_sale=True, show_real_lease=True, apt_info=None):
        """시세/실거래 차트 그리기"""
        self.fig.clf()
        gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
        self.ax = self.fig.add_subplot(gs[0, 0])
        self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)

        sise_data = chart_data.get('sise_data', [])
        sale_real_data = chart_data.get('sale_real_data', [])
        lease_real_data = chart_data.get('lease_real_data', [])

        if not (sise_data or sale_real_data or lease_real_data):
            self.ax.text(0.5, 0.5, "데이터가 없습니다.", ha='center', va='center', transform=self.ax.transAxes)
            self.canvas.draw()
            return

        prices_ax = self.ax
        ratio_ax = prices_ax.twinx()

        # 시세 데이터 플로팅
        if sise_data:
            dates = [pd.to_datetime(d['기준년월'], format='%Y%m') for d in sise_data]
            prices_ax.set_ylabel('가격 (만원)', color='blue')
            prices_ax.tick_params(axis='y', labelcolor='blue')
            if show_sise_sale:
                prices_ax.plot(dates, [d['매매일반거래가'] for d in sise_data], label='매매일반거래가', color='blue', linestyle='solid', linewidth=2, marker='None')
            if show_sise_lease:
                prices_ax.plot(dates, [d['전세일반거래가'] for d in sise_data], label='전세일반거래가', color='red', linestyle='solid', linewidth=2, marker='None')

            ratio_ax.set_ylabel('전세가율 (%)', color='green')
            ratio_ax.tick_params(axis='y', labelcolor='green')
            if show_sise_ratio:
                ratio_ax.plot(dates, [d['전세가율'] for d in sise_data], label='전세가율', color='green', linestyle=':', linewidth=2, marker='None')
        else:
            prices_ax.set_ylabel('가격 (만원)', color='blue')
            prices_ax.tick_params(axis='y', labelcolor='blue')
            ratio_ax.set_ylabel('전세가율 (%)', color='green')
            ratio_ax.tick_params(axis='y', labelcolor='green')

        # Y축 격자선 설정
        prices_ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5, color='gray')
        prices_ax.grid(True, which='minor', axis='y', linestyle=':', linewidth=0.25, color='lightgray')

        # 실거래가 데이터 플로팅 (점도표)
        if show_real_sale and sale_real_data:
            sale_real_dates = [pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in sale_real_data if d['거래금액'] is not None]
            sale_real_prices = [d['거래금액'] for d in sale_real_data if d['거래금액'] is not None]
            prices_ax.scatter(sale_real_dates, sale_real_prices, label='매매실거래', color='royalblue', marker='o', s=15, alpha=0.6)

        if show_real_lease and lease_real_data:
            lease_real_dates = [pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in lease_real_data if d['거래금액'] is not None]
            lease_real_prices = [d['거래금액'] for d in lease_real_data if d['거래금액'] is not None]
            prices_ax.scatter(lease_real_dates, lease_real_prices, label='전세실거래', color='firebrick', marker='s', s=15, alpha=0.6)

        # 하단 거래 건수 차트 그리기
        self.ax2.clear()
        bar_width = 20 # 열차트 선굵기 조절용

        sale_counts = {}
        lease_counts = {}

        if show_real_sale:
            for d in sale_real_data:
                month = d['기준년월일'][:6]
                sale_counts[month] = sale_counts.get(month, 0) + 1

        if show_real_lease:
            for d in lease_real_data:
                month = d['기준년월일'][:6]
                lease_counts[month] = lease_counts.get(month, 0) + 1

        months = sorted(set(list(sale_counts.keys()) + list(lease_counts.keys())))
        x = [pd.to_datetime(m, format='%Y%m') for m in months]
        sale_y = [sale_counts.get(m, 0) for m in months]
        lease_y = [lease_counts.get(m, 0) for m in months]

        if show_real_sale and x:
            self.ax2.bar([d - pd.Timedelta(days=bar_width/2) for d in x], sale_y, width=bar_width, color='royalblue', label='매매 건수')
        if show_real_lease and x:
            self.ax2.bar([d + pd.Timedelta(days=bar_width/2) for d in x], lease_y, width=bar_width, color='firebrick', label='전세 건수')

        self.ax2.set_ylabel('거래 건수')
        self.ax2.grid(True, which='major', axis='y')
        self.ax2.legend(loc='upper left')
        self.ax2.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

        # X축 날짜 포맷 설정
        all_dates = []
        if sise_data and (show_sise_sale or show_sise_lease or show_sise_ratio):
            all_dates.extend([pd.to_datetime(d['기준년월'], format='%Y%m') for d in sise_data])
        if sale_real_data and show_real_sale:
            all_dates.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in sale_real_data if d['거래금액'] is not None])
        if lease_real_data and show_real_lease:
            all_dates.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in lease_real_data if d['거래금액'] is not None])

        if all_dates:
            min_date = min(all_dates) - pd.Timedelta(days=30)
            max_date = max(all_dates) + pd.Timedelta(days=30)
            self.ax.set_xlim(min_date, max_date)

            self.ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
            self.ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            self.ax.grid(True, which='major', axis='x', linestyle='--', linewidth=0.5, color='gray')

            self.ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
            self.ax.grid(True, which='minor', axis='x', linestyle=':', linewidth=0.25, color='lightgray')

            self.ax2.grid(True, which='major', axis='x', linestyle='--', linewidth=0.5, color='gray')
            self.ax2.grid(True, which='minor', axis='x', linestyle=':', linewidth=0.25, color='lightgray')

            plt.setp(self.ax.get_xticklabels(), visible=False)
            self.fig.autofmt_xdate(rotation=45)
        
        # 시세차트 범례
        lines_prices, labels_prices = prices_ax.get_legend_handles_labels()
        lines_ratio, labels_ratio = ratio_ax.get_legend_handles_labels()

        all_lines_for_ax = lines_prices + lines_ratio
        all_labels_for_ax = labels_prices + labels_ratio

        order_for_ax = ['매매일반거래가', '전세일반거래가', '전세가율', '매매실거래', '전세실거래']
        handles_labels_for_ax = []
        for h, l in zip(all_lines_for_ax, all_labels_for_ax):
            if l in order_for_ax:
                handles_labels_for_ax.append((h, l))

        if handles_labels_for_ax:
            handles_labels_for_ax.sort(key=lambda hl: order_for_ax.index(hl[1]))
            handles_ax, labels_ax = zip(*handles_labels_for_ax)
            self.ax.legend(handles_ax, labels_ax, loc='upper left')

        # 차트 제목 설정
        if apt_info:
            title_text = f"{apt_info.get('단지명', '단지명 없음')} - {apt_info.get('전용면적', '면적 없음')}㎡"
        else:
            title_text = "아파트 시세 및 실거래가 변화"
        self.ax.set_title(title_text)
        
        self.fig.tight_layout()
        self.canvas.draw()

    def update_chart_summary_info(self, chart_data):
        """차트 하단 요약 정보 업데이트"""
        # 레이블 초기화
        for label_key in self.chart_info_labels:
            self.chart_info_labels[label_key].config(text=f"{label_key}: -")

        if not chart_data:
            return

        sise_data = chart_data.get('sise_data', [])
        sale_real_data = chart_data.get('sale_real_data', [])
        lease_real_data = chart_data.get('lease_real_data', [])

        # 매매 시세 최고가/최근가
        max_sale_price_sise = None
        max_sale_price_sise_date = None
        latest_sale_price_sise = None
        if sise_data:
            latest_sale_price_sise = sise_data[-1].get('매매일반거래가')
            for item in sise_data:
                price = item.get('매매일반거래가')
                if price is not None:
                    if max_sale_price_sise is None or price > max_sale_price_sise:
                        max_sale_price_sise = price
                        max_sale_price_sise_date = datetime.strptime(item['기준년월'], "%Y%m") 
        self.chart_info_labels['매매최고가'].config(text=f"매매최고가: {max_sale_price_sise if max_sale_price_sise is not None else '-'} 만원 ({max_sale_price_sise_date.strftime('%Y-%m') if max_sale_price_sise_date else '-'})")
        self.chart_info_labels['매매최근가'].config(text=f"매매최근가: {latest_sale_price_sise if latest_sale_price_sise is not None else '-'} 만원")

        # 전세 시세 최고가/최근가
        max_lease_price_sise = None
        max_lease_price_sise_date = None 
        latest_lease_price_sise = None
        if sise_data:
            latest_lease_price_sise = sise_data[-1].get('전세일반거래가')
            for item in sise_data:
                price = item.get('전세일반거래가')
                if price is not None:
                    if max_lease_price_sise is None or price > max_lease_price_sise:
                        max_lease_price_sise = price
                        max_lease_price_sise_date = datetime.strptime(item['기준년월'], "%Y%m") 
        self.chart_info_labels['전세최고가'].config(text=f"전세최고가: {max_lease_price_sise if max_lease_price_sise is not None else '-'} 만원 ({max_lease_price_sise_date.strftime('%Y-%m') if max_lease_price_sise_date else '-'})") 
        self.chart_info_labels['전세최근가'].config(text=f"전세최근가: {latest_lease_price_sise if latest_lease_price_sise is not None else '-'} 만원")

        # 실거래 매매 최고가/최근가
        max_sale_price_real = None
        max_sale_price_real_date = None
        latest_sale_price_real = None
        if sale_real_data:
            latest_sale_price_real = sale_real_data[-1].get('거래금액')
            for item in sale_real_data:
                price = item.get('거래금액')
                if price is not None:
                    if max_sale_price_real is None or price > max_sale_price_real:
                        max_sale_price_real = price
                        max_sale_price_real_date = datetime.strptime(item['기준년월일'], "%Y%m%d")
        self.chart_info_labels['실거래매매최고가'].config(text=f"실거래매매최고가: {max_sale_price_real if max_sale_price_real is not None else '-'} 만원 ({max_sale_price_real_date.strftime('%Y-%m-%d') if max_sale_price_real_date else '-'})")
        self.chart_info_labels['실거래매매최근가'].config(text=f"실거래매매최근가: {latest_sale_price_real if latest_sale_price_real is not None else '-'} 만원")

        # 실거래 전세 최고가/최근가
        max_lease_price_real = None
        max_lease_price_real_date = None
        latest_lease_price_real = None
        if lease_real_data:
            latest_lease_price_real = lease_real_data[-1].get('거래금액')
            for item in lease_real_data:
                price = item.get('거래금액')
                if price is not None:
                    if max_lease_price_real is None or price > max_lease_price_real:
                        max_lease_price_real = price
                        max_lease_price_real_date = datetime.strptime(item['기준년월일'], "%Y%m%d")
        self.chart_info_labels['실거래전세최고가'].config(text=f"실거래전세최고가: {max_lease_price_real if max_lease_price_real is not None else '-'} 만원 ({max_lease_price_real_date.strftime('%Y-%m-%d') if max_lease_price_real_date else '-'})")
        self.chart_info_labels['실거래전세최근가'].config(text=f"실거래전세최근가: {latest_lease_price_real if latest_lease_price_real is not None else '-'} 만원")

        # 매매하락가, 매매하락율, 매매회복율
        decline_price = None
        decline_rate_str = "N/A"
        recovery_rate_str = "N/A"

        if max_sale_price_sise is not None and max_sale_price_sise_date is not None and latest_sale_price_sise is not None:
            sise_after_peak_values = []
            for item in sise_data:
                item_date = datetime.strptime(item['기준년월'], "%Y%m")
                if item_date >= max_sale_price_sise_date and item.get('매매일반거래가') is not None:
                    sise_after_peak_values.append(item.get('매매일반거래가'))
            
            if sise_after_peak_values:
                decline_price = min(sise_after_peak_values)
                
                if max_sale_price_sise > 0:
                    decline_rate = ((max_sale_price_sise - decline_price) / max_sale_price_sise) * 100
                    decline_rate_str = f"{decline_rate:.2f}%"
                else:
                    decline_rate_str = "N/A"

                if decline_price is not None and max_sale_price_sise is not None and (max_sale_price_sise - decline_price) > 0:
                    if latest_sale_price_sise > decline_price and latest_sale_price_sise < max_sale_price_sise:
                        recovery_rate = ((latest_sale_price_sise - decline_price) / (max_sale_price_sise - decline_price)) * 100
                        recovery_rate_str = f"{recovery_rate:.2f}%"
                    elif latest_sale_price_sise >= max_sale_price_sise:
                        recovery_rate_str = "100.00% (최고가 회복)"
                    else:
                        recovery_rate_str = "0.00%"
                else:
                    recovery_rate_str = "N/A"
            else:
                decline_price = max_sale_price_sise
                decline_rate_str = "0.00%"
                recovery_rate_str = "N/A"#excel
                
        self.chart_info_labels['매매하락가'].config(text=f"매매하락가: {decline_price if decline_price is not None else '-'} 만원")
        self.chart_info_labels['매매하락율'].config(text=f"매매하락율: {decline_rate_str}")
        self.chart_info_labels['매매회복율'].config(text=f"매매회복율: {recovery_rate_str}")

    def clear_charts(self):
        """차트 영역 초기화"""
        self.fig.clf()
        gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
        self.ax = self.fig.add_subplot(gs[0, 0])
        self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)
        self.ax.text(0.5, 0.5, "단지를 선택하여 차트를 로드하세요.", ha='center', va='center', transform=self.ax.transAxes)
        self.canvas.draw()
        self.update_chart_summary_info(None) # 차트 요약 정보도 초기화

    def clear_info(self):
        """정보 패널 초기화"""
        for item in self.sale_real_tran_tree.get_children():
            self.sale_real_tran_tree.delete(item)
        for item in self.lease_real_tran_tree.get_children():
            self.lease_real_tran_tree.delete(item)
        self.update_chart_summary_info(None) # 차트 요약 정보 초기화

    def create_timeseries_analysis_page(self, parent_frame):
        """3페이지: 시계열 분석 UI 생성"""
        # 메인 PanedWindow (3분할: 좌측, 중앙, 우측)
        main_paned = ttk.PanedWindow(parent_frame, orient=tk.HORIZONTAL)
        main_paned.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # === 좌측 영역 (다운로드 및 설정) ===
        left_frame = ttk.Frame(main_paned, width=300)
        main_paned.add(left_frame, weight=1)

        # 좌측 상단: 다운로드 버튼 및 경로 설정
        download_frame = ttk.LabelFrame(left_frame, text="시계열 다운로드", padding=10)
        download_frame.pack(fill=tk.X, padx=5, pady=5)

        # 다운로드 버튼들
        btn_frame = ttk.Frame(download_frame)
        btn_frame.pack(fill=tk.X, pady=(0, 10))

        kb_weekly_btn = ttk.Button(btn_frame, text="KB 주간시계열", 
                                  command=self.download_kb_weekly_timeseries)
        kb_weekly_btn.pack(side=tk.LEFT, padx=(0, 5), fill=tk.X, expand=True)

        kb_monthly_btn = ttk.Button(btn_frame, text="KB 월간시계열", 
                                   command=self.download_kb_monthly_timeseries)
        kb_monthly_btn.pack(side=tk.LEFT, padx=(5, 0), fill=tk.X, expand=True)

        # 다운로드 경로 설정
        path_frame = ttk.Frame(download_frame)
        path_frame.pack(fill=tk.X, pady=(0, 5))

        ttk.Label(path_frame, text="다운로드 경로:").pack(anchor=tk.W)
        
        path_entry_frame = ttk.Frame(path_frame)
        path_entry_frame.pack(fill=tk.X, pady=(5, 0))
        
        path_entry = ttk.Entry(path_entry_frame, textvariable=self.timeseries_download_path_var)
        path_entry.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        
        path_browse_btn = ttk.Button(path_entry_frame, text="찾기", 
                                    command=self.browse_timeseries_download_path)
        path_browse_btn.pack(side=tk.RIGHT)

        # 좌측 하단: 분석 기준 선택
        analysis_frame = ttk.LabelFrame(left_frame, text="분석 설정", padding=10)
        analysis_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # 시도/시군구 선택 (1행 2열)
        region_frame = ttk.Frame(analysis_frame)
        region_frame.pack(fill=tk.X, pady=(0, 10))

        # 1열: 시도 선택
        sido_frame = ttk.Frame(region_frame)
        sido_frame.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        ttk.Label(sido_frame, text="시도:").pack(anchor=tk.W)
        self.timeseries_sido_combobox = ttk.Combobox(sido_frame, textvariable=self.timeseries_sido_var, 
                                                    state="readonly")
        self.timeseries_sido_combobox.pack(fill=tk.X, pady=(2, 0))
        self.timeseries_sido_combobox.bind("<<ComboboxSelected>>", self.on_timeseries_sido_selected)

        # 2열: 시군구 선택
        sigungu_frame = ttk.Frame(region_frame)
        sigungu_frame.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(5, 0))
        ttk.Label(sigungu_frame, text="시군구:").pack(anchor=tk.W)
        self.timeseries_sigungu_combobox = ttk.Combobox(sigungu_frame, textvariable=self.timeseries_sigungu_var, 
                                                       state="readonly")
        self.timeseries_sigungu_combobox.pack(fill=tk.X, pady=(2, 0))

        # 3열: 기간 설정
        period_frame = ttk.Frame(analysis_frame)
        period_frame.pack(fill=tk.X, pady=(0, 10))

        ttk.Label(period_frame, text="분석 기간:").pack(anchor=tk.W)
        
        period_input_frame = ttk.Frame(period_frame)
        period_input_frame.pack(fill=tk.X, pady=(2, 0))

        ttk.Label(period_input_frame, text="시작:").pack(side=tk.LEFT)
        start_entry = ttk.Entry(period_input_frame, textvariable=self.timeseries_period_start_var, width=12)
        start_entry.pack(side=tk.LEFT, padx=(5, 10))

        ttk.Label(period_input_frame, text="종료:").pack(side=tk.LEFT)
        end_entry = ttk.Entry(period_input_frame, textvariable=self.timeseries_period_end_var, width=12)
        end_entry.pack(side=tk.LEFT, padx=(5, 0))

        # === 차트 데이터 선택 영역 ===
        chart_data_frame = ttk.LabelFrame(analysis_frame, text="차트 데이터 선택", padding=10)
        chart_data_frame.pack(fill=tk.X, pady=(10, 0))

        # 체크버튼들을 3x3 그리드로 배치
        chart_options = [
            ('매수심리', '매수심리'),
            ('전세현황', '전세현황'),
            ('전세수급', '전세수급'),
            ('매매전망', '매매전망'),
            ('전세전망', '전세전망'),
            ('매매지수', '매매지수'),
            ('전세지수', '전세지수'),
            ('매매증감', '매매증감'),
            ('전세증감', '전세증감')
        ]

        for i, (key, display_text) in enumerate(chart_options):
            row = i // 3
            col = i % 3
            cb = ttk.Checkbutton(chart_data_frame, text=display_text, 
                                variable=self.chart_data_vars[key],
                                command=self.on_chart_data_selection_changed)
            cb.grid(row=row, column=col, sticky=tk.W, padx=5, pady=2)

        # 전체 선택/해제 버튼
        select_all_frame = ttk.Frame(chart_data_frame)
        select_all_frame.grid(row=3, column=0, columnspan=3, pady=(10, 0))

        ttk.Button(select_all_frame, text="전체 선택", 
                  command=self.select_all_chart_data).pack(side=tk.LEFT, padx=(0, 5))
        ttk.Button(select_all_frame, text="전체 해제", 
                  command=self.deselect_all_chart_data).pack(side=tk.LEFT)

        # 분석 실행 버튼
        analyze_btn = ttk.Button(analysis_frame, text="차트 생성", 
                               command=self.generate_timeseries_chart)
        analyze_btn.pack(fill=tk.X, pady=(10, 0))

        # === 우측 영역 (차트 표시) ===
        right_frame = ttk.Frame(main_paned)
        main_paned.add(right_frame, weight=2)

        # 차트 프레임
        self.timeseries_chart_frame = ttk.LabelFrame(right_frame, text="시계열 분석 차트", padding=10)
        self.timeseries_chart_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)

        # 초기 차트 설정
        self.setup_timeseries_chart()

        # 시도/시군구 콤보박스 초기 데이터 로드
        self.load_timeseries_region_data()

    def browse_timeseries_download_path(self):
        """시계열 다운로드 경로 선택"""
        path = tkinter.filedialog.askdirectory()
        if path:
            self.timeseries_download_path_var.set(path)

    def on_timeseries_sido_selected(self, event):
        """시계열 분석 시도 선택 이벤트 핸들러"""
        selected_sido = self.timeseries_sido_var.get()
        if selected_sido and selected_sido in self.region_data:
            sigungu_list = list(self.region_data[selected_sido].keys())
            self.timeseries_sigungu_combobox['values'] = sigungu_list
            if sigungu_list:
                self.timeseries_sigungu_var.set(sigungu_list[0])
            else:
                self.timeseries_sigungu_var.set("")
        else:
            self.timeseries_sigungu_combobox['values'] = []
            self.timeseries_sigungu_var.set("")


    def on_chart_data_selection_changed(self):
        """차트 데이터 선택 변경 이벤트 핸들러"""
        # 선택된 항목이 있는지 확인
        selected_count = sum(var.get() for var in self.chart_data_vars.values())
        
        if selected_count == 0:
            # 모든 선택이 해제된 경우 매수심리를 다시 선택
            self.chart_data_vars['매수심리'].set(True)
            messagebox.showinfo("안내", "최소 하나의 차트 데이터를 선택해야 합니다. 매수심리가 자동으로 선택되었습니다.")

    def select_all_chart_data(self):
        """모든 차트 데이터 선택"""
        for var in self.chart_data_vars.values():
            var.set(True)

    def deselect_all_chart_data(self):
        """모든 차트 데이터 선택 해제"""
        for var in self.chart_data_vars.values():
            var.set(False)

    def generate_timeseries_chart(self):
        """시계열 차트 생성 (plot_multiple_timeseries_data 호출)"""
        # 입력 값 검증
        selected_sido = self.timeseries_sido_var.get()
        selected_sigungu = self.timeseries_sigungu_var.get()
        
        if not selected_sido or not selected_sigungu:
            messagebox.showwarning("경고", "시도와 시군구를 선택해주세요.")
            return

        try:
            start_date = datetime.strptime(self.timeseries_period_start_var.get(), "%Y-%m-%d")
            end_date = datetime.strptime(self.timeseries_period_end_var.get(), "%Y-%m-%d")
            if start_date >= end_date:
                messagebox.showwarning("경고", "시작일은 종료일보다 빨라야 합니다.")
                return
        except ValueError:
            messagebox.showwarning("경고", "날짜 형식이 올바르지 않습니다 (YYYY-MM-DD).")
            return

        # 선택된 차트 데이터 확인
        selected_charts = [key for key, var in self.chart_data_vars.items() if var.get()]
        if not selected_charts:
            messagebox.showwarning("경고", "최소 하나의 차트 데이터를 선택해주세요.")
            return

        # 다운로드 경로에서 시계열 파일 찾기
        download_path = self.timeseries_download_path_var.get()
        weekly_file = os.path.join(download_path, "주간시계열.xlsx")
        monthly_file = os.path.join(download_path, "월간시계열.xlsx")

        # 파일 존재 확인
        available_files = []
        if os.path.exists(weekly_file):
            available_files.append(("주간", weekly_file))
        if os.path.exists(monthly_file):
            available_files.append(("월간", monthly_file))

        if not available_files:
            messagebox.showwarning("경고", "시계열 파일을 찾을 수 없습니다.\n먼저 시계열 데이터를 다운로드해주세요.")
            return

        # 데이터 로드 및 차트 생성
        self.loading_label.config(text="시계열 데이터를 분석하고 차트를 생성 중입니다...")
        
        try:
            self.plot_multiple_timeseries_data(available_files, selected_sido, selected_sigungu, 
                                             start_date, end_date, selected_charts)
            self.loading_label.config(text="시계열 차트가 성공적으로 생성되었습니다.")
        except Exception as e:
            messagebox.showerror("오류", f"시계열 차트 생성 중 오류가 발생했습니다:\n{str(e)}")
            self.loading_label.config(text="시계열 차트 생성에 실패했습니다.")

    def setup_timeseries_chart(self):
        """시계열 차트 초기 설정"""
        self.timeseries_fig = Figure(figsize=(8, 6), dpi=100)
        self.timeseries_ax = self.timeseries_fig.add_subplot(111)
        self.timeseries_canvas = FigureCanvasTkAgg(self.timeseries_fig, master=self.timeseries_chart_frame)
        self.timeseries_canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    def load_timeseries_region_data(self):
        """시계열 분석용 지역 데이터 로드"""
        if self.region_data:
            self.timeseries_sido_combobox['values'] = list(self.region_data.keys())

    def create_property_collector_page(self, parent_frame):
        """4페이지: 매물수집기 UI 생성"""
        # 입력 프레임 구성
        input_frame = tk.Frame(parent_frame, padx=10, pady=10, relief="groove", bd=2)
        input_frame.pack(pady=10, fill="x")

        # ----- 제1세로열: 지역 선택 콤보박스 (Column 0-1) -----
        combobox_label_col = 0 
        combobox_widget_col = 1 

        current_row_for_combobox = 0 
        tk.Label(input_frame, text="시/도 선택:", font=('맑은 고딕', 10, 'bold')).grid(row=current_row_for_combobox, column=combobox_label_col, padx=5, pady=5, sticky="w")
        self.collector_sido_combobox = ttk.Combobox(input_frame, state="readonly", font=('맑은 고딕', 10))
        self.collector_sido_combobox.grid(row=current_row_for_combobox, column=combobox_widget_col, padx=5, pady=5, sticky="ew")
        self.collector_sido_combobox.bind("<<ComboboxSelected>>", self.on_collector_sido_selected)

        current_row_for_combobox += 1
        tk.Label(input_frame, text="시/군/구 선택:", font=('맑은 고딕', 10, 'bold')).grid(row=current_row_for_combobox, column=combobox_label_col, padx=5, pady=5, sticky="w")
        self.collector_gungu_combobox = ttk.Combobox(input_frame, state="readonly", font=('맑은 고딕', 10))
        self.collector_gungu_combobox.grid(row=current_row_for_combobox, column=combobox_widget_col, padx=5, pady=5, sticky="ew")
        self.collector_gungu_combobox.bind("<<ComboboxSelected>>", self.on_collector_gungu_selected)

        current_row_for_combobox += 1
        tk.Label(input_frame, text="법정동 선택:", font=('맑은 고딕', 10, 'bold')).grid(row=current_row_for_combobox, column=combobox_label_col, padx=5, pady=5, sticky="w")
        self.collector_legal_dong_combobox = ttk.Combobox(input_frame, state="readonly", font=('맑은 고딕', 10))
        self.collector_legal_dong_combobox.grid(row=current_row_for_combobox, column=combobox_widget_col, padx=5, pady=5, sticky="ew")
        self.collector_legal_dong_combobox.bind("<<ComboboxSelected>>", self.on_collector_legal_dong_selected)

        # 법정동 코드 저장을 위한 StringVar
        self.cortar_no_var = tk.StringVar() 
        # 좌표 정보 저장용
        self.coord_vars = {
            'lat': tk.StringVar(), 'lon': tk.StringVar(), 'btm': tk.StringVar(),
            'lft': tk.StringVar(), 'top': tk.StringVar(), 'rgt': tk.StringVar()
        }

        # ----- 중앙 영역: 체크박스 그룹들 (Column 2-7) -----
        checkbox_label_start_col = 2 
        checkbox_widget_start_col = 3 

        checkbox_current_row = 0 

        # --- 부동산 유형 (체크박스 그룹) ---
        tk.Label(input_frame, text="부동산 유형:", font=('맑은 고딕', 10, 'bold')).grid(row=checkbox_current_row, column=checkbox_label_start_col, padx=10, pady=5, sticky="w")
        rlet_type_checkbox_frame = tk.Frame(input_frame)
        rlet_type_checkbox_frame.grid(row=checkbox_current_row, column=checkbox_widget_start_col, padx=5, pady=5, sticky="w")
        self.rlet_type_vars = {} 
        for name in self._RLET_TYPE_CODES.keys():
            var = tk.BooleanVar(value=(name == '아파트')) 
            cb = tk.Checkbutton(rlet_type_checkbox_frame, text=name, variable=var, font=('맑은 고딕', 10))
            cb.pack(anchor="w", side="left") 
            self.rlet_type_vars[name] = var

        # --- 거래 유형 (체크박스 그룹) ---
        checkbox_current_row += 1
        tk.Label(input_frame, text="거래 유형:", font=('맑은 고딕', 10, 'bold')).grid(row=checkbox_current_row, column=checkbox_label_start_col, padx=10, pady=5, sticky="w")
        trad_type_checkbox_frame = tk.Frame(input_frame)
        trad_type_checkbox_frame.grid(row=checkbox_current_row, column=checkbox_widget_start_col, padx=5, pady=5, sticky="w")
        self.trad_type_vars = {} 
        for name in self._TRAD_TYPE_CODES.keys():
            var = tk.BooleanVar(value=True) 
            cb = tk.Checkbutton(trad_type_checkbox_frame, text=name, variable=var, font=('맑은 고딕', 10))
            cb.pack(anchor="w", side="left") 
            self.trad_type_vars[name] = var
        
        # --- 평형대 (체크박스 그룹) ---
        checkbox_current_row += 1
        tk.Label(input_frame, text="평형대:", font=('맑은 고딕', 10, 'bold')).grid(row=checkbox_current_row, column=checkbox_label_start_col, padx=10, pady=5, sticky="w")
        pyeong_type_checkbox_frame = tk.Frame(input_frame)
        pyeong_type_checkbox_frame.grid(row=checkbox_current_row, column=checkbox_widget_start_col, padx=5, pady=5, sticky="w", rowspan=2)
        self.pyeong_type_vars = {} 
        
        # 평형대 체크박스를 2행으로 나누어 나열
        pyeong_names = list(self._PYEONG_TYPE_RANGES.keys())
        num_cols_per_row_in_frame = (len(pyeong_names) + 1) // 2 
        
        for i, name in enumerate(pyeong_names):
            row_in_frame = i // num_cols_per_row_in_frame 
            col_in_frame = i % num_cols_per_row_in_frame 
            
            var = tk.BooleanVar(value=False) 
            cb = tk.Checkbutton(pyeong_type_checkbox_frame, text=name, variable=var, font=('맑은 고딕', 10))
            cb.grid(row=row_in_frame, column=col_in_frame, padx=5, sticky="w") 
            self.pyeong_type_vars[name] = var

        # count_info_frame의 시작 행을 평형대 체크박스 그룹의 마지막 행 다음으로 설정합니다.
        # 평형대 체크박스 그룹은 checkbox_current_row부터 2행을 차지하므로, 다음 위젯은 checkbox_current_row + 2 에 옵니다.
        count_info_row_start = checkbox_current_row + 2

        count_info_frame = tk.LabelFrame(input_frame, text="거래 유형별 매물 수", padx=10, pady=5)
        # 중앙 체크박스 영역의 너비(col 2,3,4,5,6,7 총 6칸)를 커버하도록 columnspan 설정
        # 0컬럼부터 시작하면 버튼과 겹치므로, 중앙 체크박스 컬럼(checkbox_label_start_col)에서 시작하도록 변경
        count_info_frame.grid(row=count_info_row_start, column=checkbox_label_start_col, columnspan=6, padx=5, pady=10, sticky="ew") # 시작 컬럼과 columnspan 조정

        tk.Label(count_info_frame, textvariable=self.collector_sale_count_var, font=('맑은 고딕', 10, 'bold')).pack(side=tk.LEFT, padx=5)
        tk.Label(count_info_frame, textvariable=self.collector_lease_count_var, font=('맑은 고딕', 10, 'bold')).pack(side=tk.LEFT, padx=5)
        tk.Label(count_info_frame, textvariable=self.collector_monthly_count_var, font=('맑은 고딕', 10, 'bold')).pack(side=tk.LEFT, padx=5)
        tk.Label(count_info_frame, textvariable=self.collector_total_count_var, font=('맑은 고딕', 10, 'bold'), fg="blue").pack(side=tk.RIGHT, padx=5)
        # ----- 매물 개수 UI 추가 끝 -----
        
        # 단지명 필터 UI 추가
        # 이전 UI 요소들과 겹치지 않도록 적절한 row 설정
        # 예시: query_button_row - 1 (버튼 위)
        # 또는 새로운 grid row 변수 사용

        # 임시로 단지명 필터UI를 줌 레벨/최대 페이지 설정이 있던 right_section_label_col/widget_col 옆으로 옮김
        # (원래 plan에서 우측에 설정과 버튼들이 있던 위치)

        # 줌 레벨, 최대 페이지 등 설정 변수는 그대로 두되, UI는 여기에 추가
        # ----- 오른쪽 영역: 설정 및 버튼들 (Column 8-9) -----
        right_section_label_col = 8
        right_section_widget_col = 9

        # --- 이 부분을 기존 위치에서 옮기거나 다시 추가해야 합니다 ---
        save_section_start_row = 0
        tk.Label(input_frame, text="저장 경로:", font=('맑은 고딕', 10, 'bold')).grid(row=save_section_start_row, column=right_section_label_col, padx=10, pady=5, sticky="w")
        self.collector_save_path_var = tk.StringVar(value="results.csv")
        tk.Entry(input_frame, textvariable=self.collector_save_path_var, width=20, font=('맑은 고딕', 10)).grid(row=save_section_start_row, column=right_section_widget_col, padx=5, pady=5, sticky="ew")

        save_section_start_row += 1
        tk.Button(input_frame, text="저장 경로 선택", command=self.select_collector_save_path, font=('맑은 고딕', 10)).grid(row=save_section_start_row, column=right_section_label_col, columnspan=2, padx=5, pady=5, sticky="ew")

        save_section_start_row += 1
        self.collector_save_button = tk.Button(input_frame, text="조회 결과 저장", command=self.save_collector_articles_to_file, bg="#28A745", fg="white", font=('맑은 고딕', 10, 'bold'), state=tk.DISABLED)
        self.collector_save_button.grid(row=save_section_start_row, column=right_section_label_col, columnspan=2, padx=5, pady=10, sticky="ew")

        # 단지명 필터 UI 추가 (기존 우측 영역 시작 행에서 1행 추가)
        complex_filter_row = save_section_start_row + 1 # 조회 결과 저장 버튼 아래에 추가
        tk.Label(input_frame, text="단지명 필터:", font=('맑은 고딕', 10, 'bold')).grid(row=complex_filter_row, column=right_section_label_col, padx=10, pady=5, sticky="w")
        self.collector_complex_combobox = ttk.Combobox(input_frame, textvariable=self.collector_complex_name_var, state="readonly", font=('맑은 고딕', 10))
        self.collector_complex_combobox.grid(row=complex_filter_row, column=right_section_widget_col, padx=5, pady=5, sticky="ew")
        self.collector_complex_combobox.bind("<<ComboboxSelected>>", self.on_collector_complex_name_selected)

        # --- 여기에 거래유형 필터 UI 추가 ---
        trad_type_filter_row = complex_filter_row + 1 # 단지명 필터 아래에 배치
        tk.Label(input_frame, text="거래 유형 필터:", font=('맑은 고딕', 10, 'bold')).grid(row=trad_type_filter_row, column=right_section_label_col, padx=10, pady=5, sticky="w")
        self.collector_trad_type_combobox = ttk.Combobox(input_frame, textvariable=self.collector_trad_type_var, state="readonly", font=('맑은 고딕', 10))
        self.collector_trad_type_combobox.grid(row=trad_type_filter_row, column=right_section_widget_col, padx=5, pady=5, sticky="ew")
        self.collector_trad_type_combobox.bind("<<ComboboxSelected>>", self.on_collector_trad_type_selected)

        # 줌 레벨 설정 (숨김)
        self.z_level_var = tk.IntVar(value=12)
        # 최대 페이지 설정 (숨김)
        self.max_pages_var = tk.IntVar(value=500)
        
        # --- 버튼 재배치 ---
        # 매물 조회 버튼
        query_button_row = current_row_for_combobox + 1 
        self.collector_query_button = tk.Button(input_frame, text="매물 조회", command=self.start_collector_fetch_thread, bg="#007BFF", fg="white", font=('맑은 고딕', 10, 'bold'), width=10) 
        self.collector_query_button.grid(row=query_button_row, column=combobox_label_col, columnspan=combobox_widget_col + 1, padx=5, pady=10, sticky="ew") 

        # 중지 버튼
        stop_button_row = query_button_row + 1 
        self.collector_stop_button = tk.Button(input_frame, text="중지", command=self.stop_collector_fetch, bg="#FF4500", fg="white", font=('맑은 고딕', 10, 'bold'), state=tk.DISABLED, width=10) 
        self.collector_stop_button.grid(row=stop_button_row, column=combobox_label_col, columnspan=combobox_widget_col + 1, padx=5, pady=10, sticky="ew") 

        # 저장 공간 입력창 및 버튼
        save_section_start_row = 0 
        tk.Label(input_frame, text="저장 경로:", font=('맑은 고딕', 10, 'bold')).grid(row=save_section_start_row, column=right_section_label_col, padx=10, pady=5, sticky="w")
        self.collector_save_path_var = tk.StringVar(value="results.csv") 
        tk.Entry(input_frame, textvariable=self.collector_save_path_var, width=20, font=('맑은 고딕', 10)).grid(row=save_section_start_row, column=right_section_widget_col, padx=5, pady=5, sticky="ew")
        
        save_section_start_row += 1
        tk.Button(input_frame, text="저장 경로 선택", command=self.select_collector_save_path, font=('맑은 고딕', 10)).grid(row=save_section_start_row, column=right_section_label_col, columnspan=2, padx=5, pady=5, sticky="ew")

        save_section_start_row += 1
        self.collector_save_button = tk.Button(input_frame, text="조회 결과 저장", command=self.save_collector_articles_to_file, bg="#28A745", fg="white", font=('맑은 고딕', 10, 'bold'), state=tk.DISABLED)
        self.collector_save_button.grid(row=save_section_start_row, column=right_section_label_col, columnspan=2, padx=5, pady=10, sticky="ew")

        # 그리드 컬럼 설정
        input_frame.grid_columnconfigure(combobox_widget_col, weight=1) 
        input_frame.grid_columnconfigure(checkbox_widget_start_col, weight=1) 
        input_frame.grid_columnconfigure(right_section_widget_col, weight=1) 

        # 결과용 테이블
        table_frame = tk.Frame(parent_frame, padx=10, pady=10)
        table_frame.pack(fill="both", expand=True)

        columns = ("atclNo", "atclNm", "tradTpNm", "hanPrc", "prc", "rentPrc", "bildNm", "flrInfo", "spc1", "spc2", "direction", "atclFetrDesc")
        self.collector_tree = ttk.Treeview(table_frame, columns=columns, show="headings")

        headers = ["매물번호", "매물명", "거래종류", "한글가격", "매매/전세가", "월세가",
                   "동정보", "층정보", "공급(㎡)", "전용(㎡)", "방향", "특이사항"]
        
        # 매물수집기 Treeview 전용 정렬 방향 저장 딕셔너리 추가
        self.collector_sort_directions = {} # 이 한 줄을 클래스 초기화(__init__) 시점에 추가해도 좋습니다.

        # 각 컬럼의 너비 설정 (필요에 따라 조정)
        column_widths = {
            "atclNo": 90, "atclNm": 320, "tradTpNm": 90, "hanPrc": 120, "rentPrc": 100,
            "bildNm": 100, "flrInfo": 100, "spc1": 100, "spc2": 100, "direction": 100,
            "atclFetrDesc": 550 # 특이사항은 내용이 길 수 있으므로 너비를 넓게 설정
        }

        for col, text in zip(columns, headers):
            self.collector_tree.heading(col, text=text, 
                                        command=lambda _col=col: self._sort_column(self.collector_tree, self.collector_sort_directions, _col)) # 수정된 부분
            self.collector_tree.column(col, width=column_widths.get(col, 100), anchor="center", stretch=tk.NO)

        vsb = ttk.Scrollbar(table_frame, orient="vertical", command=self.collector_tree.yview)
        vsb.pack(side="right", fill="y")
        self.collector_tree.configure(yscrollcommand=vsb.set)

        hsb = ttk.Scrollbar(table_frame, orient="horizontal", command=self.collector_tree.xview)
        hsb.pack(side="bottom", fill="x")
        self.collector_tree.configure(xscrollcommand=hsb.set)

        self.collector_tree.pack(fill="both", expand=True)
        self.collector_tree.bind("<Double-1>", self._open_naver_article_webpage) # <Double-1>은 마우스 왼쪽 버튼 더블클릭 이벤트입니다.

        self.status_label = tk.Label(parent_frame, text="", bd=1, relief="sunken", anchor="w")
        self.status_label.pack(side="bottom", fill="x")

        # 지역 데이터 저장용
        self._collector_regions_data = {
            'sido': {},
            'gungu': {},
            'legal_dong': {}
        }

        # 앱 시작 후 시도 목록 자동 로드
        self.root.after(100, self.load_collector_sido_list) 

    # 매물수집기 관련 메서드들
    
    def load_collector_sido_list(self): 
        """네이버 부동산 시도 목록 로드"""
        self.update_status("시/도 목록 로드 중...")
        
        url = GET_REGION_LIST_API_URL 
        params = {"cortarNo": "0000000000", "mycortarNo": ""}
        
        data = self._safe_request('get', url, NAVER_HEADERS, params=params, api_name="지역 목록 API - 시도") 
        
        if data and 'result' in data and 'list' in data['result']:
            sido_list_raw = data['result']['list']
            
            sido_names = []
            self._collector_regions_data['sido'] = {} 
            for item in sido_list_raw:
                sido_name = item.get('CortarNm')
                sido_code = item.get('CortarNo')
                if sido_name and sido_code:
                    sido_names.append(sido_name)
                    self._collector_regions_data['sido'][sido_name] = sido_code
            
            self.collector_sido_combobox['values'] = sido_names
            
            self.collector_sido_combobox.unbind("<<ComboboxSelected>>")
            if sido_names:
                self.collector_sido_combobox.set(sido_names[0]) 
                self.on_collector_sido_selected(None)
            else:
                self.collector_sido_combobox.set("시도 선택")
            self.collector_sido_combobox.bind("<<ComboboxSelected>>", self.on_collector_sido_selected)
            self.update_status("시/도 목록 로드 완료.")
        else:
            self.update_status("⛔️ 시/도 목록 로드 실패.")
            self.root.after(0, lambda: messagebox.showerror("오류", "시도 목록을 가져오지 못했습니다.")) 
            self.collector_sido_combobox['values'] = ["목록 로드 실패"]
            self.collector_sido_combobox.set("목록 로드 실패")
        self.root.update_idletasks()

    def on_collector_sido_selected(self, event):
        selected_sido_name = self.collector_sido_combobox.get()
        selected_sido_code = self._collector_regions_data['sido'].get(selected_sido_name)
        
        if not selected_sido_code:
            return

        self.update_status(f"'{selected_sido_name}'의 시군구 목록 로드 중...")
        self.collector_gungu_combobox.set('')
        self.collector_gungu_combobox['values'] = []
        self.collector_legal_dong_combobox.set('')
        self.collector_legal_dong_combobox['values'] = []
        self._collector_regions_data['gungu'] = {} 
        self._collector_regions_data['legal_dong'] = {} 

        url = GET_REGION_LIST_API_URL
        params = {"cortarNo": selected_sido_code, "mycortarNo": ""}
        
        response_data = self._safe_request('get', url, NAVER_HEADERS, params=params, api_name="지역 목록 API - 시군구")
        
        if response_data and 'result' in response_data and 'list' in response_data['result']:
            gungu_list_raw = response_data['result']['list']
            gungu_names = []
            for gungu_data in gungu_list_raw:
                name = gungu_data.get('CortarNm')
                code = gungu_data.get('CortarNo')
                if name and code:
                    gungu_names.append(name)
                    self._collector_regions_data['gungu'][name] = {'code': code}
            self.collector_gungu_combobox['values'] = gungu_names
            self.update_status(f"'{selected_sido_name}'의 시군구 목록 로드 완료.")
        else:
            self.update_status(f"⛔️ '{selected_sido_name}'의 시군구 목록 로드 실패.")
            self.root.after(0, lambda: messagebox.showerror("오류", f"'{selected_sido_name}' 시군구 목록을 가져오지 못했습니다."))

    def on_collector_gungu_selected(self, event):
        selected_gungu_name = self.collector_gungu_combobox.get()
        gungu_info = self._collector_regions_data['gungu'].get(selected_gungu_name)
        
        if not gungu_info:
            return

        selected_gungu_code = gungu_info.get('code')

        self.update_status(f"'{selected_gungu_name}'의 법정동 목록 로드 중...")
        self.collector_legal_dong_combobox.set('')
        self.collector_legal_dong_combobox['values'] = []
        self._collector_regions_data['legal_dong'] = {}

        url = GET_REGION_LIST_API_URL
        params = {"cortarNo": selected_gungu_code, "mycortarNo": ""}

        response_data = self._safe_request('get', url, NAVER_HEADERS, params=params, api_name="지역 목록 API - 법정동")

        if response_data and 'result' in response_data and 'list' in response_data['result']:
            legal_dong_list_raw = response_data['result']['list']
            legal_dong_names = []
            for legal_dong_data in legal_dong_list_raw:
                name = legal_dong_data.get('CortarNm')
                code = legal_dong_data.get('CortarNo')
                if name and code:
                    legal_dong_names.append(name)
                    self._collector_regions_data['legal_dong'][name] = {
                        'code': code,
                        'lat_center': float(legal_dong_data.get('MapYCrdn')),
                        'lon_center': float(legal_dong_data.get('MapXCrdn')),
                    }
            self.collector_legal_dong_combobox['values'] = legal_dong_names
            self.update_status(f"'{selected_gungu_name}'의 법정동 목록 로드 완료.")
        else:
            self.update_status(f"⛔️ '{selected_gungu_name}'의 법정동 목록 로드 실패.")
            self.root.after(0, lambda: messagebox.showerror("오류", f"'{selected_gungu_name}' 법정동 목록을 가져오지 못했습니다."))

    def on_collector_legal_dong_selected(self, event):
        selected_legal_dong_name = self.collector_legal_dong_combobox.get()
        legal_dong_info = self._collector_regions_data['legal_dong'].get(selected_legal_dong_name)

        if not legal_dong_info:
            return
        
        self.cortar_no_var.set(legal_dong_info.get('code', '')) 
        
        center_lat = legal_dong_info.get('lat_center')
        center_lon = legal_dong_info.get('lon_center')

        if center_lat is not None and center_lon is not None:
            try:
                z_level = int(self.z_level_var.get())
            except ValueError:
                z_level = 14

            deltas = self._Z_LEVEL_DELTAS.get(z_level, self._Z_LEVEL_DELTAS[14]) 
            lat_delta = deltas['lat_delta']
            lon_delta = deltas['lon_delta']

            btm_val = center_lat - lat_delta
            top_val = center_lat + lat_delta
            lft_val = center_lon - lon_delta
            rgt_val = center_lon + lon_delta
            
            self.coord_vars['lat'].set(str(center_lat))
            self.coord_vars['lon'].set(str(center_lon))
            self.coord_vars['btm'].set(str(btm_val))
            self.coord_vars['lft'].set(str(lft_val))
            self.coord_vars['top'].set(str(top_val))
            self.coord_vars['rgt'].set(str(rgt_val))
            
            self.update_status(f"'{selected_legal_dong_name}' 선택 완료. 좌표 자동 입력됨.")
        else:
            for var in self.coord_vars.values():
                var.set('')
            self.update_status(f"'{selected_legal_dong_name}' 선택 완료. 중심 좌표 정보가 없어 자동 입력 불가.")

    def start_collector_fetch_thread(self):
        if hasattr(self, 'collector_fetch_thread') and self.collector_fetch_thread.is_alive():
            self.root.after(0, lambda: messagebox.showwarning("경고", "이미 매물 조회 작업이 진행 중입니다."))
            return
        self._stop_flag = False
        self.root.after(0, lambda: self.collector_query_button.config(state=tk.DISABLED))
        self.root.after(0, lambda: self.collector_stop_button.config(state=tk.NORMAL))
        self.collector_save_button.config(state=tk.DISABLED)
        self.update_status("매물 조회 시작...")
        self.fetched_article_data = []

        self.collector_fetch_thread = threading.Thread(target=self.fetch_collector_articles, daemon=True)
        self.collector_fetch_thread.start()

    def stop_collector_fetch(self):
        self._stop_flag = True
        self.update_status("중지 요청됨. 현재 페이지 작업 완료 후 중단됩니다.")
        self.root.after(0, lambda: self.collector_stop_button.config(state=tk.DISABLED))

    def _add_articles_to_collector_tree(self, articles_to_add):
        """지정된 매물들을 매물수집기 Treeview에 추가합니다."""
        inserted_count_current_page = 0
        for art in articles_to_add:
            tree_values = (
                art.get('atclNo', ''),
                art.get('atclNm', ''),
                art.get('tradTpNm', ''),
                art.get('hanPrc', ''),
                art.get('prc', ''),
                art.get('rentPrc', ''),
                art.get('bildNm', ''),
                art.get('flrInfo', ''),
                art.get('spc1', ''),
                art.get('spc2', ''),
                art.get('direction', ''),
                art.get('atclFetrDesc', '')
            )
            item_id_for_iid = str(art.get('atclNo'))
            
            # Treeview에 항목 삽입 (GUI 스레드에서 안전하게 실행)
            # lambda 함수는 for 루프 바깥에서 art를 캡처하도록 합니다.
            # Treeview.insert 호출 시 tag 옵션은 없습니다. (별도의 색상 태그를 여기서 줄 수 있으나, 현재 요청에선 없음)
#            self.collector_tree.insert("", "end", values=tree_values, iid=item_id_for_iid)
            # 🟢 추가: 이미 존재하는지 확인
            if not self.collector_tree.exists(item_id_for_iid):
                self.collector_tree.insert("", "end", values=tree_values, iid=item_id_for_iid)
            else:
                print(f"중복 항목 스킵 - iid: {item_id_for_iid}")
            inserted_count_current_page += 1
        
        # 총 매물 수 업데이트는 _apply_all_collector_filters에서 일괄 처리되므로 여기서는 Treeview 업데이트만 집중
        # (아니면 여기서 실시간으로 count_var를 업데이트해도 됩니다, 취사 선택)

        print(f"DEBUG: Treeview에 {inserted_count_current_page}개 매물 추가 완료.")

    # 또한, 매물 수집 완료 후 호출되는 _apply_all_collector_filters 내부의 Treeview 채우는 부분은 제거해야 합니다.
    # 왜냐하면 이미 _add_articles_to_collector_tree에서 매물을 페이지마다 채웠기 때문입니다.
    
    def fetch_collector_articles(self):
        """
        네이버 부동산 API를 통해 매물 데이터를 실제로 조회합니다.
        페이지마다 데이터를 가져와 Treeview에 실시간으로 업데이트합니다.
        500개 매물마다 긴 대기 시간을 가집니다.
        """

        # 매물 조회 시작 전, 모든 매물 데이터를 저장할 변수 초기화
        self.all_fetched_articles = []  # 기존 데이터 초기화
        self.fetched_article_data = []  # 페이지별 표시용 데이터 초기화

        # 🟢 추가: Treeview도 완전히 초기화
        if hasattr(self, 'collector_tree') and self.collector_tree:
            for item in self.collector_tree.get_children():
                self.collector_tree.delete(item)
        
        cortar_no = self.cortar_no_var.get().strip()
        max_pages = self.max_pages_var.get()

        # 원본 좌표값 저장 (페이지마다 미세 조정 시 기준점)
        original_lat = float(self.coord_vars['lat'].get().strip())
        original_lon = float(self.coord_vars['lon'].get().strip())
        original_btm = float(self.coord_vars['btm'].get().strip())
        original_lft = float(self.coord_vars['lft'].get().strip())
        original_top = float(self.coord_vars['top'].get().strip())
        original_rgt = float(self.coord_vars['rgt'].get().strip())

        # 부동산 유형 체크박스 상태를 기반으로 최종 rletTpCd 생성
        selected_rlet_types = []
        for name, var in self.rlet_type_vars.items():
            if var.get():
                api_code_str = self._RLET_TYPE_CODES.get(name, '')
                if api_code_str:
                    selected_rlet_types.extend(api_code_str.split(':'))
        rlet_tp_cd_final = ":".join(sorted(list(set(selected_rlet_types))))

        # 거래 유형 체크박스 상태를 기반으로 최종 tradTpCd 생성
        selected_trad_types = []
        for name, var in self.trad_type_vars.items():
            if var.get():
                api_code = self._TRAD_TYPE_CODES.get(name, '')
                if api_code:
                    selected_trad_types.append(api_code)
        trad_tp_cd_final = ":".join(sorted(list(set(selected_trad_types))))

        # 평형대 체크박스 상태를 기반으로 spcMin/spcMax 생성
        min_spc = float('inf')
        max_spc = 0.0
        is_pyeong_selected = False
        for name, var in self.pyeong_type_vars.items():
            if var.get():
                is_pyeong_selected = True
                pyeong_range = self._PYEONG_TYPE_RANGES[name]
                min_val = pyeong_range.get('spcMin', 0)
                max_val = pyeong_range.get('spcMax', float('inf'))

                min_spc = min(min_spc, min_val)
                max_spc = max(max_spc, max_val)
        
        params_spcMin = None
        params_spcMax = None
        if is_pyeong_selected:
            params_spcMin = str(int(min_spc)) if min_spc != float('inf') else None
            params_spcMax = str(int(max_spc)) if max_spc != float('inf') else None

        # 필수 입력값 유효성 검사
        if not cortar_no:
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "법정동을 선택하세요."))
            self.reset_collector_buttons_state()
            return
        if max_pages <= 0:
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "최대 페이지 수는 1 이상이어야 합니다."))
            self.reset_collector_buttons_state()
            return
        if not rlet_tp_cd_final:
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "하나 이상의 부동산 유형을 선택해주세요."))
            self.reset_collector_buttons_state()
            return
        if not trad_tp_cd_final:
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "하나 이상의 거래 유형을 선택해주세요."))
            self.reset_collector_buttons_state()
            return
        if not is_pyeong_selected:
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "하나 이상의 평형대를 선택해주세요."))
            self.reset_collector_buttons_state()
            return
        if not (original_lat and original_lon and original_btm and original_lft and original_top and original_rgt):
            self.root.after(0, lambda: messagebox.showwarning("입력 오류", "선택된 법정동의 지도 좌표 정보가 불완전합니다."))
            self.reset_collector_buttons_state()
            return

        total_fetched_count_overall = 0 # 전체 조회된 매물 수 (500개 대기 기준)
        self.update_status("매물 조회 중...")

        for page in range(1, max_pages + 1):
            if self._stop_flag: # 사용자의 중지 요청 확인
                self.update_status("사용자 요청으로 조회 중단됨.")
                break

            try:
                z_level = int(self.z_level_var.get())
            except ValueError:
                z_level = 14 # 기본 줌 레벨

            # 매 50페이지(예시)마다 좌표를 미세 조정하여 요청 다양화
            current_lat = original_lat
            current_lon = original_lon
            current_btm = original_btm
            current_top = original_top
            current_lft = original_lft
            current_rgt = original_rgt

            if page % 50 == 1 and page > 1: # 1페이지가 아닌 50페이지 단위의 시작 페이지 (예: 51, 101, ...)
                # 각 좌표에 랜덤한 작은 값 더하거나 빼기 (-0.001 ~ 0.001)
                random_lat_offset = random.uniform(-0.001, 0.001)
                random_lon_offset = random.uniform(-0.001, 0.001)
                
                current_lat = original_lat + random_lat_offset
                current_lon = original_lon + random_lon_offset
                
                # 변경된 중심 좌표를 기반으로 바운딩 박스 다시 계산
                lat_delta = self._Z_LEVEL_DELTAS.get(z_level, self._Z_LEVEL_DELTAS[14])['lat_delta']
                lon_delta = self._Z_LEVEL_DELTAS.get(z_level, self._Z_LEVEL_DELTAS[14])['lon_delta']
                current_btm = current_lat - lat_delta
                current_top = current_lat + lat_delta
                current_lft = current_lon - lon_delta
                current_rgt = current_lon + lon_delta
                
                print(f"DEBUG: 페이지 {page}에서 좌표 변경 적용: (lat:{current_lat:.4f}, lon:{current_lon:.4f})")
            
            # API 요청 파라미터 구성
            params = {
                "itemId": "", "mapKey": "", "lgeo": "", "showR0": "",
                "rletTpCd": rlet_tp_cd_final,
                "tradTpCd": trad_tp_cd_final,
                "z": str(z_level),
                "lat": str(current_lat), "lon": str(current_lon),
                "btm": str(current_btm), "lft": str(current_lft),
                "top": str(current_top), "rgt": str(current_rgt),
                "totCnt": "0", # 이 값은 API 응답으로 채워짐
                "cortarNo": cortar_no,
                "sort": "rank", # 정렬 기준
                "page": page # 현재 페이지 번호
            }
            # 평형대 필터가 선택된 경우에만 파라미터 추가
            if params_spcMin is not None:
                params['spcMin'] = params_spcMin
            if params_spcMax is not None:
                params['spcMax'] = params_spcMax
            
            # API 호출
            response = self._safe_request('get', GET_API_URL, NAVER_HEADERS, params=params, api_name=f"매물 조회 API (Page {page})")

            if response == "RETRY": # 429 에러 등으로 재시도 요청 시
                page -= 1 # 현재 페이지를 재시도하기 위해 페이지 번호 감소
                continue # 다음 루프 (현재 페이지 재시도)

            # API 응답 유효성 검사
            articles_on_current_page = response.get('body', []) if response else [] # 현재 페이지의 매물 리스트

            if not articles_on_current_page: # 현재 페이지에 매물이 더 없으면 조회 중단
                self.update_status(f"Page {page}: 더 이상 매물이 존재하지 않습니다. 조회 종료.")
                break
            
            # 페이지마다 조회된 매물을 Treeview에 실시간으로 추가
            self.root.after(0, lambda arts_list=articles_on_current_page: self._add_articles_to_collector_tree(arts_list))
            
            # 모든 원본 매물 데이터에 현재 페이지 매물 추가
            self.all_fetched_articles.extend(articles_on_current_page) 
            total_fetched_count_overall += len(articles_on_current_page) # 전체 매물 수 누적

            # --- 500개 매물마다 30~60초 대기 추가 ---
            # 다음 500개 매물 경계 도달 예상 지점 계산
            next_500_threshold = ((total_fetched_count_overall - 1) // 500 + 1) * 500
            
            # 현재 수집된 매물 수가 다음 500개 경계에 도달했거나 넘어섰을 경우 (또는 첫 500개 완료 시)
            # and 다음 페이지 조회를 시도하기 전
            if total_fetched_count_overall >= 500 and (total_fetched_count_overall % 500 == 0 or total_fetched_count_overall >= next_500_threshold):
                # 마지막 페이지가 아니라면 긴 대기
                if page < max_pages: 
                    delay_long = random.uniform(30, 60) # 30~60초 랜덤 대기
                    self.update_status(f"현재 {total_fetched_count_overall}개 매물 수집 완료. API 과부하 방지를 위해 {delay_long:.1f}초 대기 중...")
                    time.sleep(delay_long)
            else: # 일반적인 페이지 사이 짧은 딜레이
                delay_short = random.uniform(0, 1) # 0~1초 랜덤 대기
                self.update_status(f"Page {page} 완료. {len(articles_on_current_page)}개 매물 추가. 총 {total_fetched_count_overall}개. 다음 페이지 조회 전 {delay_short:.1f}초 대기 중...")
                time.sleep(delay_short)

            if self._stop_flag: # 대기 후에도 중지 요청이 있다면 다시 확인
                self.update_status("사용자 요청으로 조회 중단됨.")
                break
        
        # 모든 조회 완료 후 (또는 중단 후) 최종 처리
        self.update_status(f"총 {total_fetched_count_overall}개의 매물 조회를 완료했습니다. 단지명/거래유형 필터를 적용 중...")
        
        # 필터 콤보박스 업데이트 및 필터 적용 (이것은 GUI 스레드에서 실행되도록 after() 사용)
        self.root.after(0, self._update_collector_filters_and_apply) 

        self.reset_collector_buttons_state() # 조회/중지 버튼 상태 리셋
        self.root.after(0, lambda: messagebox.showinfo("조회 완료", f"총 {total_fetched_count_overall}개의 매물을 가져왔습니다."))

    def _update_complex_filter_combobox(self):
        """매물 조회 후, 수집된 전체 매물에서 고유 단지명 목록을 추출하여 단지명 필터 콤보박스 업데이트"""
        self.unique_complex_names = []
        if self.all_fetched_articles:
            # 모든 매물에서 'atclNm' (매물명)만 추출하여 고유한 값들을 세트로 만들고 리스트로 변환 후 정렬
            names = set([art.get('atclNm', '') for art in self.all_fetched_articles if art.get('atclNm')])
            self.unique_complex_names = sorted(list(names))

        # 콤보박스에 표시될 값 준비: "전체" 옵션 포함
        combobox_values = ["전체"] + self.unique_complex_names
    
        self.root.after(0, lambda: self.collector_complex_combobox.config(values=combobox_values))
        self.root.after(0, lambda: self.collector_complex_name_var.set("전체")) # 기본 선택은 "전체"

    def on_collector_complex_name_selected(self, event):
        """단지명 필터 콤보박스 선택 시 매물 Treeview 필터링 및 업데이트"""
        self._apply_all_collector_filters()

    def _update_trad_type_filter_combobox(self):
        """수집된 매물에서 고유 거래유형 추출하여 거래유형 필터 콤보박스 업데이트"""
        self.unique_trad_types = []
        if self.all_fetched_articles:
            # 'tradTpNm' (거래종류)만 추출하여 고유한 값들을 세트로 만들고 리스트로 변환 후 정렬
            names = set([art.get('tradTpNm', '') for art in self.all_fetched_articles if art.get('tradTpNm')])
            self.unique_trad_types = sorted(list(names))

        # 콤보박스에 표시될 값 준비: "전체" 옵션 포함
        combobox_values = ["전체"] + self.unique_trad_types

        self.root.after(0, lambda: self.collector_trad_type_combobox.config(values=combobox_values))
        self.root.after(0, lambda: self.collector_trad_type_var.set("전체")) # 기본 선택은 "전체"

    def on_collector_trad_type_selected(self, event):
        """거래유형 필터 콤보박스 선택 시 통합 필터링 메서드 호출"""
        self._apply_all_collector_filters()

    def _update_collector_filters_and_apply(self):
        """컬렉터 필터 UI를 업데이트하고 모든 필터를 적용합니다."""
        self._update_complex_filter_combobox()
        self._update_trad_type_filter_combobox()
        self._apply_all_collector_filters() # 모든 필터 적용 (초기 "전체"로)

    def _apply_all_collector_filters(self):
        """
        현재 선택된 모든 필터(단지명, 거래유형 등)를 적용하여 매물 Treeview 업데이트
        매물 개수 집계 및 저장 버튼 상태 업데이트 포함
        """
        # 기존 Treeview 내용 비우기 (GUI 스레드에서 안전하게 실행)
        self.root.after(0, lambda: [self.collector_tree.delete(i) for i in self.collector_tree.get_children()])
        # print(f"DEBUG: _apply_all_collector_filters 시작. all_fetched_articles 초기 수: {len(self.all_fetched_articles)}") # 변경: 디버그 출력 제거

        filtered_articles = self.all_fetched_articles # 원본 데이터를 필터링 시작점으로 사용

        # 1. 단지명 필터 적용
        selected_complex = self.collector_complex_name_var.get()
        if selected_complex != "전체":
            filtered_articles = [art for art in filtered_articles if art.get('atclNm', '') == selected_complex]
        # print(f"DEBUG: 단지명 필터 '{selected_complex}' 적용 후 수: {len(filtered_articles)}") # 변경: 디버그 출력 제거

        # 2. 거래유형 필터 적용
        selected_trad_type = self.collector_trad_type_var.get()
        if selected_trad_type != "전체":
            filtered_articles = [art for art in filtered_articles if art.get('tradTpNm', '') == selected_trad_type]
        # print(f"DEBUG: 거래유형 필터 '{selected_trad_type}' 적용 후 최종 수: {len(filtered_articles)}") # 변경: 디버그 출력 제거

        # 최종 필터링된 데이터를 self.fetched_article_data에 저장
        self.fetched_article_data = filtered_articles

        # ----- 여기에 거래 유형별 매물 개수 집계 로직 추가 -----
        sale_count = 0
        lease_count = 0
        monthly_count = 0
        
        for art in self.fetched_article_data:
            trad_type = art.get('tradTpNm', '')
            if "매매" in trad_type:
                sale_count += 1
            elif "전세" in trad_type:
                lease_count += 1
            elif "월세" in trad_type:
                monthly_count += 1
        
        total_count = len(self.fetched_article_data) # 최종 필터링된 총 매물 수
        
        # UI 라벨 업데이트 (GUI 스레드에서 안전하게)
        self.root.after(0, lambda: self.collector_sale_count_var.set(f"매매: {sale_count}개"))
        self.root.after(0, lambda: self.collector_lease_count_var.set(f"전세: {lease_count}개"))
        self.root.after(0, lambda: self.collector_monthly_count_var.set(f"월세: {monthly_count}개"))
        self.root.after(0, lambda: self.collector_total_count_var.set(f"총: {total_count}개"))
        # ----- 거래 유형별 매물 개수 집계 끝 -----

        # print(f"DEBUG: Treeview 삽입 시작. 삽입할 매물 수: {len(self.fetched_article_data)}") # 변경: 디버그 출력 제거
        
        # 필터링된 데이터를 Treeview에 다시 채우기
        # _add_articles_to_collector_tree 메서드는 UI 스레드에서 실행되므로 안전하게 호출
        self.root.after(0, lambda data_to_add=self.fetched_article_data: self._add_articles_to_collector_tree(data_to_add))
        
        # print(f"DEBUG: Treeview 채우기 요청 완료.") # 변경: 디버그 출력 제거

        # UI 하단 상태 메시지 업데이트
        self.update_status(f"필터링 결과: {len(self.fetched_article_data)}개 매물 표시 중.")

        # 저장 버튼 상태 업데이트: 필터링된 데이터가 있으면 활성화
        self.root.after(0, lambda: self.collector_save_button.config(state=tk.NORMAL if self.fetched_article_data else tk.DISABLED))
        
        # UI 강제 업데이트 (필수 아닐 수 있으나 즉각 반영을 위해 유지)
        self.root.update_idletasks()

    def _add_articles_to_collector_tree(self, articles_to_add):
        """
        지정된 매물들을 매물수집기 Treeview에 추가합니다.
        fetch_collector_articles 내부에서 페이지마다 호출됩니다.
        """
        for art in articles_to_add:
            tree_values = (
                art.get('atclNo', ''), # 매물 번호
                art.get('atclNm', ''), # 매물명
                art.get('tradTpNm', ''), # 거래종류
                art.get('hanPrc', ''), # 한글가격
                art.get('prc', ''), # 매매/전세가
                art.get('rentPrc', ''),     # 월세가
                art.get('bildNm', ''),      # 동정보
                art.get('flrInfo', ''),     # 층정보
                art.get('spc1', ''), # 공급(㎡)
                art.get('spc2', ''), # 전용(㎡)
                art.get('direction', ''),   # 방향
                art.get('atclFetrDesc', '') # 특이사항
            )
            item_id_for_iid = str(art.get('atclNo'))

            # Treeview에 항목 삽입 (GUI 스레드에서 안전하게 실행)
            self.collector_tree.insert("", "end", values=tree_values, iid=item_id_for_iid)
        self.root.update_idletasks() # UI 강제 업데이트

    def _open_naver_article_webpage(self, event):
        """매물 목록 Treeview 항목을 더블클릭했을 때 네이버 부동산 웹페이지를 엽니다."""
        tree_widget = event.widget 
        selected_item_id = tree_widget.focus() # 더블클릭된 항목의 iid (articleNo가 될 것)

        print(f"더블클릭 이벤트 발생. selected_item_id: {selected_item_id}") # 디버깅 출력

        if selected_item_id:
            article_no = selected_item_id # Treeview의 iid가 articleNo로 넘어와야 함

            # 'I001'과 같은 잘못된 값이 아닌, 실제 유효한 매물 번호인지 확인
            if article_no and str(article_no).isdigit(): # article_no가 비어있지 않고, 숫자로만 구성되어 있는지 확인
                article_url = f"https://fin.land.naver.com/articles/{article_no}"
                
                try:
                    webbrowser.open(article_url)
                    print(f"웹페이지 열기 시도: {article_url}")
                    self.update_debug_label(f"매물 웹페이지 열림: {article_url}")
                except Exception as e:
                    messagebox.showerror("웹페이지 열기 오류", f"웹 브라우저를 열 수 없습니다: {e}")
                    self.update_debug_label(f"웹페이지 열기 오류: {e}")
            else:
                messagebox.showwarning("정보 없음", f"선택된 매물 항목의 웹페이지 정보(매물 번호)가 유효하지 않습니다: '{article_no}'")
                self.update_debug_label(f"유효하지 않은 매물 번호: {article_no}")
        else:
            print("선택된 매물 항목이 없습니다.")
            self.update_debug_label("매물 항목 미선택.")

    def reset_collector_buttons_state(self):
        self.root.after(0, lambda: self.collector_query_button.config(state=tk.NORMAL))
        self.root.after(0, lambda: self.collector_stop_button.config(state=tk.DISABLED))

    def select_collector_save_path(self):
        selected_dong_name = self.collector_legal_dong_combobox.get()
        if not selected_dong_name:
            selected_dong_name = "매물"
        
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        default_filename = f"{selected_dong_name}_{timestamp}.csv"

        file_path = tkinter.filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("All files", "*.*")],
            initialfile=default_filename
        )
        if file_path:
            self.collector_save_path_var.set(file_path)

    def save_collector_articles_to_file(self):
        if not self.fetched_article_data:
            messagebox.showwarning("저장 오류", "저장할 매물 데이터가 없습니다.")
            return
        
        file_path = self.collector_save_path_var.get()
        if not file_path:
            messagebox.showwarning("저장 오류", "저장 경로를 입력해주세요.")
            return

        try:
            df = pd.DataFrame(self.fetched_article_data)
            
            # 매핑 정의
            column_mapping = {
                'atclNo': "매물번호", 'atclNm': "매물명", 'tradTpNm': "거래종류", 
                'hanPrc': "한글가격", 'prc': "매매/전세가", 'rentPrc': "월세가", 
                'flrInfo': "층정보", 'spc1': "공급(㎡)", 'spc2': "전용(㎡)", 
                'direction': "방향", 'cortarNo': "법정동코드"
            }
            # DataFrame에서 필요한 컬럼만 추출하고 순서 정렬
            cols_to_extract = list(column_mapping.keys())
            df_filtered = df[cols_to_extract].copy()
            # 컬럼명 변경
            df_filtered = df_filtered.rename(columns=column_mapping)
            
            # CSV로 저장 (UTF-8 인코딩)
            df_filtered.to_csv(file_path, index=False, encoding='utf-8-sig') 
            messagebox.showinfo("저장 완료", f"매물 데이터가 '{file_path}'에 성공적으로 저장되었습니다.")
            self.update_status(f"데이터를 '{file_path}'에 저장했습니다.")
        except Exception as e:
            messagebox.showerror("저장 오류", f"파일 저장 중 오류가 발생했습니다: {e}")
            self.update_status(f"⛔️ 저장 오류: {e}")

    # 프로그램 실행 함수
    def run(self):
        self.root.mainloop()

# 프로그램 시작 시 모든 만료일 검사 수행 (최종 통합본)
if __name__ == "__main__":
    current_date = date.today() # date.today() 사용
    # 이제 이 프로그램의 최종 만료일은 EXPIRATION_DATE가 됩니다.
    # 프로그램 인스턴스가 생성되기 전에 미리 모든 검사를 마쳐야 합니다.

    if current_date > EXPIRATION_DATE:
        messagebox.showerror("프로그램 만료", f"프로그램 사용 기한이 {EXPIRATION_DATE}로 만료되었습니다.\n더 이상 프로그램을 사용할 수 없습니다.")
        sys.exit() # 즉시 종료

    # 만료 예정일 알림 (예: 30일 전)
    expiration_warning_date = EXPIRATION_DATE - timedelta(days=30) # timedelta 사용
    
    if current_date >= expiration_warning_date and current_date <= EXPIRATION_DATE:
        remaining_days = (EXPIRATION_DATE - current_date).days
        messagebox.showwarning(
            "사용 기한 임박",
            f"⚠️ 프로그램 사용 기한이 {EXPIRATION_DATE}까지입니다. (남은 기간: {remaining_days}일)\n"
            "미리 준비해 주세요."
        )

    # 모든 검사를 통과했을 때만 애플리케이션 인스턴스 생성 및 실행
    # PROGRAM_EXPIRATION_DATE_STR은 이제 필요 없는 상수이므로,
    # InvestmentTableProgram 생성자에 다른 인자가 필요 없다면 제거하거나,
    # 실제 앱 내부에서 참조할 만료일 변수가 필요하면 EXPIRATION_DATE를 직접 전달합니다.
    app = InvestmentTableProgram() # EXPIRATION_DATE를 문자열로 전달 (만약 app 내에서 이 값이 필요하다면)
    app.run()

pywin32 라이브러리를 성공적으로 로드했습니다.
DPI 인식 설정을 시도했습니다.
[GitHub]에서 지역코드 파일 로드 시도: https://raw.githubusercontent.com/kaga-fo/DaonSearcher_Project/refs/heads/main/%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C.txt
[GitHub]에서 지역코드 파일 성공적으로 로드.
지역코드 파싱 완료. 총 17개의 시도 데이터.

--- API 요청 시작: 지역 목록 API - 시도 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '0000000000', 'mycortarNo': ''}
  Status Code: 200

--- API 요청 시작: 지역 목록 API - 시군구 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '1100000000', 'mycortarNo': ''}
  Status Code: 200

--- API 요청 시작: 지역 목록 API - 시군구 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '4400000000', 'mycortarNo': ''}
  Status Code: 200

--- API 요청 시작: 지역 목록 API - 법정동 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '4420000000', 'mycortarNo': ''}
  Status Code: 200

--- API 요청 시작: 매물 조회 API (Page 1) ---
  URL: https://m